<a href="https://colab.research.google.com/github/baminro/Portfolio/blob/main/BLRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import datetime
import math
from numpy.random import choice
import random

from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model
from keras.regularizers import l2
import keras
from keras import layers
import numpy as np
import pandas as pd

import random
from collections import deque
import matplotlib.pylab as plt
import matplotlib.dates as mdates # Formatting dates
from sklearn.covariance import LedoitWolf
import numpy as np

In [ ]:
#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
directory = r'/content/drive/MyDrive/Adjusted'
df=[]
files_in_directory = os.listdir(directory)
filtered_files = [file for file in files_in_directory if file.endswith(".csv")]
for file in filtered_files:
    path_to_file = os.path.join(directory, file)
    aux=pd.read_csv(path_to_file, encoding="utf-16")
    df.append(aux)
df=pd.concat(df)
df['<TICKER>']= df['<TICKER>'].str.replace('-ت', '')
df['<COL12>']= df['<COL12>'].str.replace('-ت', '')
df['<COL12>']= df['<COL12>'].str.replace('_', ' ')
df['<TICKER>']= df['<TICKER>'].str.replace('_', ' ')


df.to_excel('/content/drive/MyDrive/out.xlsx')
import datetime
from datetime import datetime as dt
df['<DTYYYYMMDD>']=df['<DTYYYYMMDD>'].apply(lambda x : dt.strptime(str(x), '%Y%m%d'))


#df['<DTYYYYMMDD>'] = pd.to_datetime(df['<DTYYYYMMDD>']).dt.date
#df['<DTYYYYMMDD>'] = df['<DTYYYYMMDD>'].dt.strftime('%Y%m%d')
df=df.set_index('<DTYYYYMMDD>')
df

In [ ]:

df['SMAclose5']=df.groupby('<COL11>')['<CLOSE>'].transform(lambda x: x.rolling(5).mean())
df['SMAclose30']=df.groupby('<COL11>')['<CLOSE>'].transform(lambda x: x.rolling(30).mean())
df['EMAclose5']=df.groupby('<COL11>')['<CLOSE>'].transform(lambda x: x.ewm(span =5, min_periods = 5 - 1).mean())
df['EMAclose30']=df.groupby('<COL11>')['<CLOSE>'].transform(lambda x: x.ewm(span =30, min_periods = 5 - 1).mean())
MA = df['SMAclose5']
SD = df.groupby('<COL11>')['<CLOSE>'].transform(lambda x: x.rolling(window=5).std().mean())
#df['MiddleBandclose'] = MA
df['UpperBandclose'] = MA + (2 * SD)
df['LowerBandclose'] = MA - (2 * SD)

df['FIclose'] =df.groupby('<COL11>')['<CLOSE>'].transform(lambda x: x.diff(1)) * df['<VOL>']

df['dm'] = ((df['<HIGH>'] + df['<LOW>'])/2) - ((df.groupby('<COL11>')['<HIGH>'].transform(lambda x: x.shift(1)) + df.groupby('<COL11>')['<LOW>'].transform(lambda x: x.shift(1)))/2)
df['br'] = (df['<VOL>'] / 100000000) / ((df['<HIGH>'] - df['<LOW>']))
df['EMV'] = df['dm']/ df['br']
df['EMV_MAclose'] =df.groupby('<COL11>')['EMV'].transform(lambda x: x.rolling(5).mean())
del df['dm']
del df['br']


def rsi_transform(price_series, window=5):
    delta = price_series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    # Use EMA smoothing (standard for RSI)
    avg_gain = gain.ewm(alpha=1/window, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/window, adjust=False).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# 3. Calculate RSI by ticker using transform
df['RSIclose'] = (df.groupby('<COL11>')['<CLOSE>']
                .transform(rsi_transform, window=5))

df

In [ ]:
import numpy as np
import pandas as pd

# Log-return of 'close' per stock
df['log_returnC'] = df.groupby('<COL11>')['<CLOSE>'].transform(lambda x: np.log(x / x.shift(1)))
df['log_returnO'] = df.groupby('<COL11>')['<OPEN>'].transform(lambda x: np.log(x / x.shift(1)))
df['log_returnH'] = df.groupby('<COL11>')['<HIGH>'].transform(lambda x: np.log(x / x.shift(1)))
df['log_returnL'] = df.groupby('<COL11>')['<LOW>'].transform(lambda x: np.log(x / x.shift(1)))
# Log-change of 'volume' per stock (optional)
df['log_volume_change'] = df.groupby('<COL11>')['<VOL>'].transform(lambda x: np.log(x / x.shift(1)))


# Replace inf/-inf with NaN
dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

# Option 1: Drop rows with NaNs
dataset.dropna(inplace=True)

# Option 2: Fill NaNs (if preserving rows)
dataset.fillna(0, inplace=True)
numeric_df = dataset.select_dtypes(include=[np.number])
print(np.isinf(dataset).sum().sum())

# Check for NaNs
print(dataset.isna().sum().sum())

# Check for very large values
print((np.abs(dataset) > 1e308).sum().sum())

# Fill missing values (first row in each group)
df[['log_returnC','log_returnO','log_returnH','log_returnL', 'log_volume_change']] = df[['log_returnC','log_returnO','log_returnH','log_returnL', 'log_volume_change']]


In [ ]:
df=df.reset_index()
df=df[['<DTYYYYMMDD>','<CLOSE>','<OPEN>',	'<HIGH>',	'<LOW>','<VOL>','<COL11>','<TICKER>','SMAclose5',	'SMAclose30',	'EMAclose5',	'EMAclose30',	'UpperBandclose',	'LowerBandclose',	'FIclose',	'EMV_MAclose',	'RSIclose']]
df.columns = ['Date','<CLOSE>','<OPEN>','<HIGH>',	'<LOW>','<VOL>','<COL11>','<TICKER>','SMAclose5',	'SMAclose30',	'EMAclose5',	'EMAclose30',	'UpperBandclose',	'LowerBandclose',	'FIclose',	'EMV_MAclose', 'RSIclose']
df=df[df['Date']<='2025-05-17']

In [ ]:
df=df.pivot(index='Date', columns='<COL11>', values=['<CLOSE>','<OPEN>',	'<HIGH>',	'<LOW>','<VOL>','SMAclose5',	'UpperBandclose',	'LowerBandclose',	'FIclose',	'EMV_MAclose',	'RSIclose'])

df=df.fillna(method='ffill')

In [ ]:
col = ('<CLOSE>', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('<OPEN>', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('<HIGH>', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('<LOW>', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('<VOL>', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]





"""col = ('log_returnC', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('log_returnO', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('log_returnH', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('log_returnL', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('log_volume_change', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]"""


col = ('SMAclose5', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
'''col = ('SMAclose30', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('EMAclose5', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('EMAclose30', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]'''

col = ('UpperBandclose', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('LowerBandclose', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('FIclose', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
col = ('EMV_MAclose', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]

col = ('RSIclose', 'IDXS')
new_order = [c for c in df.columns if c != col] + [col]
df = df[new_order]
df

In [ ]:
# peek at data
set_option('display.width', 100)
dataset=dataset.reset_index()
dataset['Date'] = pd.to_datetime(dataset['Date'],utc=True)
dataset['Date'] = dataset['Date'].dt.strftime('%Y-%m-%d')
dataset=dataset.set_index('Date')
#del dataset['index']
dataset.head(5)

In [ ]:
dataindex=dataset[[('<CLOSE>', 'IDXS')]]
dataindex
dataindex.to_csv('/content/drive/MyDrive/indexnew.csv')

In [ ]:
dataset.to_csv('/content/drive/MyDrive/crypto_portfolionew.csv')


In [ ]:
import numpy as np
import pandas as pd
import os

from IPython.core.debugger import set_trace
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
import cvxpy as cp
#define a function portfolio
def portfolio(returns, weights):
    weights = np.array(weights)
    rets = returns.mean() * 252
    retsmean=returns.mean()
    rets12=returns-retsmean
    lenx=rets12.shape[0]
    rf=0.21/252
    """for i in range(lenx):
            if (rets12.iloc[i]>0):
                rets12.loc[:][i]=0"""

    rets12[rets12 > 0] = 0
    semicov=((rets12.T.dot(rets12))/(lenx-1))*252
    #covs = returns.cov() * 252
    P_ret = rets
    P_vol =returns.std()* np.sqrt(252)
    P_semivol =semicov* np.sqrt(252)
    P_sharpe = (P_ret-rf*252) / P_vol
    P_sortino = (P_ret-rf*252) / P_semivol
    ## 1. Calmar Ratio
    cumulative_returns = (returns + 1).cumprod()
    running_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - running_max) / running_max
    max_drawdown = drawdown.min()

    # daily → annualized compound return
    annualized_return = (cumulative_returns.iloc[-1]) ** (252 / len(cumulative_returns)) - 1
    calmar_ratio = (annualized_return-rf*252) / abs(max_drawdown) if max_drawdown != 0 else np.nan

    ## 2. Excess Return to Value-at-Risk (VaR)
    excess_return = returns.mean() - rf
    var_95 = np.percentile(returns, 5)
    excess_return_to_var = excess_return*252 / abs(var_95) if var_95 != 0 else np.nan

    # 3. Excess Return to Conditional Value-at-Risk (CVaR)
    excess_return = returns.mean() - rf
    var_95 = np.percentile(returns, 5)
    cvar_95 = returns[returns <= var_95].mean()
    excess_return_to_cvar = excess_return*252 / abs(cvar_95) if cvar_95 != 0 else np.nan

    return np.array([P_ret, P_vol,P_semivol, P_sharpe,P_sortino, calmar_ratio, excess_return_to_var,excess_return_to_cvar])



"""def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]           # shape: (seq_len, 1)
    i = np.arange(d_model)[np.newaxis, :]            # shape: (1, d_model)
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    # Apply sin to even indices, cos to odd indices
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]       # shape: (1, seq_len, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)"""


# -------------------------
# Positional encoding helper
# -------------------------
"""def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]           # (seq_len, 1)
    i = np.arange(d_model)[np.newaxis, :]             # (1, d_model)
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]        # (1, seq_len, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)


# -------------------------
# Subclassed transformer
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len, d_model=64, num_heads=4, dff=128, num_layers=2, dropout=0.1, name="transformer_multioutput"):
        super().__init__(name=name)
        self.seq_len = seq_len
        self.d_model = d_model
        self.n_assets = n_assets
        self.pos_encoding = get_positional_encoding(seq_len, d_model)

        # input projection
        self.input_proj = layers.Dense(d_model, name="input_projection")

        # encoder layers (store layers in lists but give each layer explicit name)
        self.attn_layers = []
        self.norm1_layers = []
        self.ffn_layers = []
        self.norm2_layers = []
        for i in range(num_layers):
            attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model, name=f"multihead_attention_{i}")
            norm1 = layers.LayerNormalization(epsilon=1e-6, name=f"attn_norm_{i}")
            ffn = tf.keras.Sequential([
                layers.Dense(dff, activation="relu", name=f"ff_{i}_dense_1"),
                layers.Dense(d_model, name=f"ff_{i}_dense_2")
            ], name=f"ffn_{i}")
            norm2 = layers.LayerNormalization(epsilon=1e-6, name=f"ffn_norm_{i}")
            self.attn_layers.append(attn)
            self.norm1_layers.append(norm1)
            self.ffn_layers.append(ffn)
            self.norm2_layers.append(norm2)

        # heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, training=False):
        # inputs shape: (batch, seq_len, n_assets)
        x = self.input_proj(inputs)                    # (batch, seq_len, d_model)
        x = x + self.pos_encoding[:, :self.seq_len, :] # broadcasting positional encoding

        for i, (attn, norm1, ffn, norm2) in enumerate(zip(self.attn_layers, self.norm1_layers, self.ffn_layers, self.norm2_layers)):
            attn_out = attn(x, x)                      # self-attention
            x = norm1(x + attn_out)                    # residual + norm
            ffn_out = ffn(x)
            x = norm2(x + ffn_out)                     # residual + norm

        # use last time step representation for heads (can be changed if desired)
        last = x[:, -1, :]                             # (batch, d_model)
        mean_out = self.mean_head(last)
        var_out = self.var_head(last)
        return {"mean_head": mean_out, "var_head": var_out}

    # keep compile signature convenient
    def compile(self, optimizer='adam', loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {'mean_head': 'mse', 'var_head': 'mse'}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)


# -------------------------
# Builder wrapper (keeps same naming across train/eval)
# -------------------------
def build_transformer_for_env(env, n_assets=18, seq_len=60, d_model=64, num_heads=4, dff=128, num_layers=2, dropout=0.1, model_path=None):
    model = TransformerModel(n_assets=n_assets, seq_len=seq_len, d_model=d_model, num_heads=num_heads, dff=dff, num_layers=num_layers, dropout=dropout)
    # build with dummy input
    dummy = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy)
    # compile
    model.compile(optimizer='adam', loss={'mean_head': 'mse', 'var_head': 'mse'}, metrics={'mean_head':'mae','var_head':'mae'})
    env.transformer_model = model

    # Optionally load weights safely
    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded transformer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load transformer weights from {model_path}: {e}\n→ Continuing with freshly initialized transformer.")"""






# -------------------------
# Positional encoding helper
# -------------------------
"""def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]           # (seq_len, 1)
    i = np.arange(d_model)[np.newaxis, :]             # (1, d_model)
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]        # (1, seq_len, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)


# -------------------------
# Simple Graph Convolution Layer
# -------------------------
class GraphConvolution(tf.keras.layers.Layer):
    def __init__(self, output_dim, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.w = self.add_weight(
            shape=(input_dim, self.output_dim),
            initializer="glorot_uniform",
            trainable=True,
            name="gcn_weight"
        )

    def call(self, inputs, adjacency_matrix):
        # inputs: [batch, n_assets, features]
        support = tf.matmul(inputs, self.w)                         # (batch, n_assets, output_dim)
        output = tf.matmul(adjacency_matrix, support)               # (batch, n_assets, output_dim)
        if self.activation is not None:
            output = self.activation(output)
        return output


# -------------------------
# GCN + Transformer hybrid model
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len, node_features=1,
                 d_model=64, num_heads=4, dff=128, num_layers=2,
                 gcn_units=32, dropout=0.1, name="gcn_transformer"):
        super().__init__(name=name)
        self.seq_len = seq_len
        self.d_model = d_model
        self.n_assets = n_assets
        self.node_features = node_features
        self.pos_encoding = get_positional_encoding(seq_len, d_model)

        # --- Graph convolution ---
        self.gcn1 = GraphConvolution(gcn_units, activation="relu", name="gcn1")
        self.gcn2 = GraphConvolution(gcn_units, activation="relu", name="gcn2")

        # projection before transformer
        self.input_proj = layers.Dense(d_model, name="input_projection")

        # --- Transformer encoder stack ---
        self.attn_layers, self.norm1_layers, self.ffn_layers, self.norm2_layers = [], [], [], []
        for i in range(num_layers):
            attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model, name=f"multihead_attention_{i}")
            norm1 = layers.LayerNormalization(epsilon=1e-6, name=f"attn_norm_{i}")
            ffn = tf.keras.Sequential([
                layers.Dense(dff, activation="relu", name=f"ff_{i}_dense_1"),
                layers.Dense(d_model, name=f"ff_{i}_dense_2")
            ], name=f"ffn_{i}")
            norm2 = layers.LayerNormalization(epsilon=1e-6, name=f"ffn_norm_{i}")
            self.attn_layers.append(attn)
            self.norm1_layers.append(norm1)
            self.ffn_layers.append(ffn)
            self.norm2_layers.append(norm2)

        # --- Prediction heads ---
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, adjacency_matrix=None, training=False):
        # If input is 3D [batch, seq_len, n_assets], expand to 4D
        if len(inputs.shape) == 3:
            inputs = tf.expand_dims(inputs, -1)  # -> [batch, seq_len, n_assets, 1]

        if adjacency_matrix is None:
            adjacency_matrix = tf.eye(self.n_assets)

        batch_size = tf.shape(inputs)[0]
        x_gcn = []

        # apply GCN at each time step
        for t in range(self.seq_len):
            x_t = inputs[:, t, :, :]                  # [batch, n_assets, node_features]
            x_t = self.gcn1(x_t, adjacency_matrix)
            x_t = self.gcn2(x_t, adjacency_matrix)
            x_gcn.append(x_t)

        # stack time dimension
        x = tf.stack(x_gcn, axis=1)                   # [batch, seq_len, n_assets, gcn_units]

        # flatten assets for transformer
        x = tf.reshape(x, [batch_size, self.seq_len, self.n_assets * x.shape[-1]])
        x = self.input_proj(x)

        # add positional encoding
        x += self.pos_encoding[:, :self.seq_len, :]

        # transformer encoder
        for attn, norm1, ffn, norm2 in zip(self.attn_layers, self.norm1_layers, self.ffn_layers, self.norm2_layers):
            attn_out = attn(x, x)
            x = norm1(x + attn_out)
            ffn_out = ffn(x)
            x = norm2(x + ffn_out)

        # final step representation
        last = x[:, -1, :]                            # [batch, d_model]
        mean_out = self.mean_head(last)
        var_out = self.var_head(last)
        return {"mean_head": mean_out, "var_head": var_out}


    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)


# -------------------------
# Builder wrapper
# -------------------------
def build_transformer_for_env(env, n_assets=18, seq_len=60, node_features=1,
                                  d_model=64, num_heads=4, dff=128, num_layers=2,
                                  gcn_units=32, dropout=0.1, model_path=None):
    model = TransformerModel(n_assets=n_assets, seq_len=seq_len, node_features=node_features,
                           d_model=d_model, num_heads=num_heads, dff=dff,
                           num_layers=num_layers, gcn_units=gcn_units, dropout=dropout)
    dummy_x = tf.zeros((1, seq_len, n_assets, node_features))
    _ = model(dummy_x)  # build
    model.compile(optimizer="adam", loss={"mean_head": "mse", "var_head": "mse"},
                  metrics={"mean_head": "mae", "var_head": "mae"})
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded GCN+Transformer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights from {model_path}: {e}")"""




# -------------------------
# Simple Graph Convolution Layer (same semantics as before)
# -------------------------
"""class GraphConvolution(tf.keras.layers.Layer):
    def __init__(self, output_dim, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        self.w = self.add_weight(
            shape=(input_dim, self.output_dim),
            initializer="glorot_uniform",
            trainable=True,
            name="gcn_weight"
        )

    def call(self, inputs, adjacency_matrix):
        # inputs: [batch, n_assets, features]
        support = tf.linalg.matmul(inputs, self.w)            # (batch, n_assets, output_dim)
        output = tf.linalg.matmul(adjacency_matrix, support)  # (batch, n_assets, output_dim)
        if self.activation is not None:
            output = self.activation(output)
        return output


# -------------------------
# Small residual TCN block (1D dilated conv)
# -------------------------
class TCNBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, dilation_rate=1, dropout=0.0, name=None):
        super().__init__(name=name)
        self.conv = layers.Conv1D(filters, kernel_size, padding="causal", dilation_rate=dilation_rate)
        self.bn = layers.BatchNormalization()
        self.act = layers.Activation("relu")
        self.dropout = layers.Dropout(dropout) if dropout and dropout > 0 else None
        self.res_conv = None  # set later if projection needed
        self.filters = filters

    def build(self, input_shape):
        in_filters = int(input_shape[-1])
        if in_filters != self.filters:
            self.res_conv = layers.Conv1D(self.filters, kernel_size=1, padding="same")

    def call(self, x, training=False):
        y = self.conv(x)
        y = self.bn(y, training=training)
        y = self.act(y)
        if self.dropout:
            y = self.dropout(y, training=training)
        if self.res_conv is not None:
            x = self.res_conv(x)
        return layers.Add()([x, y])


# -------------------------
# GCN -> LSTM -> TCN hybrid model
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len, node_features=1,
                 gcn_units=[32, 32], lstm_units=64,
                 tcn_filters=64, tcn_kernel=3, tcn_blocks=2,
                 **kwargs):
        super().__init__(**kwargs)
        self.n_assets = n_assets
        self.seq_len = seq_len
        self.node_features = node_features

        # GCN layers (corrected)
        self.gcn_layers = [GraphConvolution(units, activation="relu") for units in gcn_units]

        # LSTM
        self.lstm = layers.LSTM(lstm_units, return_sequences=True)

        # TCN (stacked Conv1D blocks)
        self.tcn_blocks = []
        for i in range(tcn_blocks):
            self.tcn_blocks.append(
                layers.Conv1D(filters=tcn_filters, kernel_size=tcn_kernel,
                              dilation_rate=2**i, padding="causal", activation="relu")
            )

        # Prediction heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, adjacency_matrix=None, training=False):
        # Accept 3D or 4D input
        if inputs.shape.rank == 3:
            inputs = tf.expand_dims(inputs, -1)  # (batch, seq, n_assets, 1)

        if adjacency_matrix is None:
            adjacency_matrix = tf.eye(self.n_assets)

        batch_size = tf.shape(inputs)[0]
        gcn_out = []

        # Apply GCN at each time step
        for t in range(self.seq_len):
            x_t = inputs[:, t, :, :]  # (batch, n_assets, features)
            for gcn in self.gcn_layers:
                x_t = gcn(x_t, adjacency_matrix)
            gcn_out.append(x_t)

        # Stack over time
        x = tf.stack(gcn_out, axis=1)  # (batch, seq, n_assets, gcn_units)
        x = tf.reshape(x, [batch_size, self.seq_len, -1])  # flatten assets

        # LSTM
        x = self.lstm(x)

        # TCN
        for conv in self.tcn_blocks:
            x = conv(x)

        # Use last time step
        last = x[:, -1, :]
        mean_out = self.mean_head(last)
        var_out = self.var_head(last)
        return {"mean_head": mean_out, "var_head": var_out}


    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)


# -------------------------
# Builder wrapper for env (keeps naming, weight loading)
# -------------------------
def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              node_features=1,
                              gcn_units=[32, 32],
                              lstm_units=64,
                              tcn_filters=64,
                              tcn_kernel=3,
                              tcn_blocks=2,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        node_features=node_features,
        gcn_units=gcn_units,
        lstm_units=lstm_units,
        tcn_filters=tcn_filters,
        tcn_kernel=tcn_kernel,
        tcn_blocks=tcn_blocks
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets, node_features))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded GCN+LSTM+TCN weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights from {model_path}: {e}")"""


# -------------------------
# Simple Graph Convolution Layer
# -------------------------
"""class GraphConvolution(tf.keras.layers.Layer):
    def __init__(self, output_dim, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        self.w = self.add_weight(
            shape=(input_dim, self.output_dim),
            initializer="glorot_uniform",
            trainable=True,
            name="gcn_weight"
        )

    def call(self, inputs, adjacency_matrix):
        # inputs: [batch, n_assets, features]
        support = tf.linalg.matmul(inputs, self.w)            # (batch, n_assets, output_dim)
        output = tf.linalg.matmul(adjacency_matrix, support)  # (batch, n_assets, output_dim)
        if self.activation is not None:
            output = self.activation(output)
        return output


# -------------------------
# Pure GNN Model
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len, node_features=1,
                 gcn_units=[32, 32], **kwargs):
        super().__init__(**kwargs)
        self.n_assets = n_assets
        self.seq_len = seq_len
        self.node_features = node_features

        # Stack of GCN layers
        self.gcn_layers = [GraphConvolution(units, activation="relu") for units in gcn_units]

        # Prediction heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, adjacency_matrix=None, training=False):
        # Accept 3D or 4D input
        if inputs.shape.rank == 3:
            inputs = tf.expand_dims(inputs, -1)  # (batch, seq, n_assets, 1)

        if adjacency_matrix is None:
            adjacency_matrix = tf.eye(self.n_assets)

        batch_size = tf.shape(inputs)[0]
        gcn_out = []

        # Apply GCN at each time step
        for t in range(self.seq_len):
            x_t = inputs[:, t, :, :]  # (batch, n_assets, features)
            for gcn in self.gcn_layers:
                x_t = gcn(x_t, adjacency_matrix)
            gcn_out.append(x_t)

        # Stack across time
        x = tf.stack(gcn_out, axis=1)  # (batch, seq, n_assets, gcn_units)

        # Flatten assets + time for final dense layer
        x = tf.reshape(x, [batch_size, self.seq_len * self.n_assets * x.shape[-1]])

        # Prediction heads
        mean_out = self.mean_head(x)
        var_out = self.var_head(x)
        return {"mean_head": mean_out, "var_head": var_out}

    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)


# -------------------------
# Builder wrapper for env
# -------------------------
def build_transformer_for_env(env,
                      n_assets=18,
                      seq_len=60,
                      node_features=1,
                      gcn_units=[32, 32],
                      model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        node_features=node_features,
        gcn_units=gcn_units
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets, node_features))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded GNN weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights (architecture mismatch). Starting fresh.")"""




# -------------------------
# TCN residual block
# -------------------------
"""class TCNBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, dilation_rate=1, dropout=0.0, name=None):
        super().__init__(name=name)
        self.conv = layers.Conv1D(filters, kernel_size,
                                  padding="causal",
                                  dilation_rate=dilation_rate)
        self.bn = layers.BatchNormalization()
        self.act = layers.Activation("relu")
        self.dropout = layers.Dropout(dropout) if dropout and dropout > 0 else None
        self.res_conv = None
        self.filters = filters

    def build(self, input_shape):
        in_filters = int(input_shape[-1])
        if in_filters != self.filters:
            self.res_conv = layers.Conv1D(self.filters, kernel_size=1, padding="same")

    def call(self, x, training=False):
        y = self.conv(x)
        y = self.bn(y, training=training)
        y = self.act(y)
        if self.dropout:
            y = self.dropout(y, training=training)
        if self.res_conv is not None:
            x = self.res_conv(x)
        return layers.Add()([x, y])


# -------------------------
# Pure TCN Model
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len,
                 tcn_filters=64, tcn_kernel=3, tcn_blocks=3,
                 dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.n_assets = n_assets
        self.seq_len = seq_len

        # TCN stack
        self.tcn_blocks = []
        for i in range(tcn_blocks):
            self.tcn_blocks.append(
                TCNBlock(filters=tcn_filters,
                         kernel_size=tcn_kernel,
                         dilation_rate=2**i,
                         dropout=dropout)
            )

        # Prediction heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, training=False):
        # Accept (batch, seq, n_assets) or (batch, seq, n_assets, 1)
        if inputs.shape.rank == 4:
            inputs = tf.squeeze(inputs, -1)  # (batch, seq, n_assets)

        # Flatten assets → treat them as features per timestep
        x = tf.reshape(inputs, [tf.shape(inputs)[0], self.seq_len, -1])  # (batch, seq, n_assets)

        # Pass through stacked TCN blocks
        for block in self.tcn_blocks:
            x = block(x, training=training)

        # Use last time step
        last = x[:, -1, :]
        mean_out = self.mean_head(last)
        var_out = self.var_head(last)
        return {"mean_head": mean_out, "var_head": var_out}

    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)


# -------------------------
# Builder wrapper for env
# -------------------------
def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              tcn_filters=64,
                              tcn_kernel=3,
                              tcn_blocks=3,
                              dropout=0.1,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        tcn_filters=tcn_filters,
        tcn_kernel=tcn_kernel,
        tcn_blocks=tcn_blocks,
        dropout=dropout
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded TCN weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights (architecture mismatch). Starting fresh.")"""



# -------------------------
# TCN residual block
# -------------------------
class TCNBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, dilation_rate=1, dropout=0.0, name=None):
        super().__init__(name=name)
        self.conv = layers.Conv1D(filters, kernel_size,
                                  padding="causal",
                                  dilation_rate=dilation_rate)
        self.bn = layers.BatchNormalization()
        self.act = layers.Activation("relu")
        self.dropout = layers.Dropout(dropout) if dropout and dropout > 0 else None
        self.res_conv = None
        self.filters = filters

    def build(self, input_shape):
        in_filters = int(input_shape[-1])
        if in_filters != self.filters:
            self.res_conv = layers.Conv1D(self.filters, kernel_size=1, padding="same")

    def call(self, x, training=False):
        y = self.conv(x)
        y = self.bn(y, training=training)
        y = self.act(y)
        if self.dropout:
            y = self.dropout(y, training=training)
        if self.res_conv is not None:
            x = self.res_conv(x)
        return layers.Add()([x, y])


# -------------------------
# TCN + LSTM Model
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len,
                 tcn_filters=64, tcn_kernel=3, tcn_blocks=3,
                 lstm_units=64, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.n_assets = n_assets
        self.seq_len = seq_len

        # TCN stack
        self.tcn_blocks = []
        for i in range(tcn_blocks):
            self.tcn_blocks.append(
                TCNBlock(filters=tcn_filters,
                         kernel_size=tcn_kernel,
                         dilation_rate=2**i,
                         dropout=dropout)
            )

        # LSTM after TCN
        self.lstm = layers.LSTM(lstm_units, return_sequences=False)

        # Prediction heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, training=False):
        # Accept (batch, seq, n_assets) or (batch, seq, n_assets, 1)
        if inputs.shape.rank == 4:
            inputs = tf.squeeze(inputs, -1)  # (batch, seq, n_assets)

        # Flatten assets → treat them as features per timestep
        x = tf.reshape(inputs, [tf.shape(inputs)[0], self.seq_len, -1])  # (batch, seq, features)

        # Pass through TCN stack
        for block in self.tcn_blocks:
            x = block(x, training=training)

        # LSTM layer (captures sequential dependencies after TCN)
        x = self.lstm(x)

        # Predictions
        mean_out = self.mean_head(x)
        var_out = self.var_head(x)
        return {"mean_head": mean_out, "var_head": var_out}

    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)


# -------------------------
# Builder wrapper for env
# -------------------------
def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              tcn_filters=64,
                              tcn_kernel=3,
                              tcn_blocks=3,
                              lstm_units=64,
                              dropout=0.1,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        tcn_filters=tcn_filters,
        tcn_kernel=tcn_kernel,
        tcn_blocks=tcn_blocks,
        lstm_units=lstm_units,
        dropout=dropout
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded TCN+LSTM weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights (architecture mismatch). Starting fresh.")




# -------------------------
# Positional Encoding
# -------------------------
"""def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]           # (seq_len, 1)
    i = np.arange(d_model)[np.newaxis, :]             # (1, d_model)
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]        # (1, seq_len, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)

# -------------------------
# ProbSparse Attention
# -------------------------
class ProbSparseAttention(layers.Layer):
    def __init__(self, d_model, num_heads, dropout=0.1, name=None):
        super().__init__(name=name)
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dropout = layers.Dropout(dropout)
        self.norm = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=False):
        attn_out = self.mha(x, x)
        attn_out = self.dropout(attn_out, training=training)
        return self.norm(x + attn_out)

# -------------------------
# Distilling Layer (Conv1D)
# -------------------------
class DistillLayer(layers.Layer):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.conv = layers.Conv1D(filters=d_model, kernel_size=3, strides=2, padding="same", activation="relu")
        self.dropout = layers.Dropout(dropout)
        self.norm = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=False):
        x = self.conv(x)
        x = self.dropout(x, training=training)
        return self.norm(x)

# -------------------------
# Informer Encoder Layer
# -------------------------
class InformerEncoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super().__init__()
        self.attn = ProbSparseAttention(d_model, num_heads, dropout)
        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation="relu"),
            layers.Dense(d_model)
        ])
        self.dropout = layers.Dropout(dropout)
        self.norm = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=False):
        x = self.attn(x, training=training)
        ffn_out = self.ffn(x)
        ffn_out = self.dropout(ffn_out, training=training)
        return self.norm(x + ffn_out)

# -------------------------
# Full Informer Model
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len,
                 d_model=64, num_heads=4, dff=128, num_layers=2,
                 dropout=0.1, name="informer_multioutput"):
        super().__init__(name=name)
        self.n_assets = n_assets
        self.seq_len = seq_len
        self.pos_encoding = get_positional_encoding(seq_len, d_model)

        # input projection
        self.input_proj = layers.Dense(d_model, name="input_projection")

        # encoder with distilling
        self.enc_layers = []
        for i in range(num_layers):
            self.enc_layers.append(InformerEncoderLayer(d_model, num_heads, dff, dropout))
            if i < num_layers - 1:
                self.enc_layers.append(DistillLayer(d_model, dropout))

        # heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, training=False):
        x = self.input_proj(inputs)
        x += self.pos_encoding[:, :self.seq_len, :]
        for layer in self.enc_layers:
            x = layer(x, training=training)
        last = x[:, -1, :]
        mean_out = self.mean_head(last)
        var_out = self.var_head(last)
        return {"mean_head": mean_out, "var_head": var_out}

    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)

# -------------------------
# Builder wrapper for env
# -------------------------
def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              d_model=64,
                              num_heads=4,
                              dff=128,
                              num_layers=2,
                              dropout=0.1,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        num_layers=num_layers,
        dropout=dropout
    )

    dummy = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded Informer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load Informer weights from {model_path}: {e}\n→ Starting fresh.")"""






# -------------------------
# Positional Encoding
# -------------------------
"""def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]          # (seq_len, 1)
    i = np.arange(d_model)[np.newaxis, :]            # (1, d_model)
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]       # (1, seq_len, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)

# -------------------------
# Simple Informer-like model
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len, d_model=64, num_heads=4, dff=128, num_layers=2, dropout=0.1):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        self.n_assets = n_assets

        # positional encoding
        self.pos_encoding = get_positional_encoding(seq_len, d_model)

        # input projection
        self.input_proj = layers.Dense(d_model, name="input_projection")

        # encoder stack
        self.attn_layers = []
        self.norm1_layers = []
        self.ffn_layers = []
        self.norm2_layers = []

        for i in range(num_layers):
            attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model, name=f"multihead_attention_{i}")
            norm1 = layers.LayerNormalization(epsilon=1e-6, name=f"attn_norm_{i}")
            ffn = tf.keras.Sequential([
                layers.Dense(dff, activation="relu", name=f"ff_{i}_dense_1"),
                layers.Dense(d_model, name=f"ff_{i}_dense_2")
            ])
            norm2 = layers.LayerNormalization(epsilon=1e-6, name=f"ffn_norm_{i}")
            self.attn_layers.append(attn)
            self.norm1_layers.append(norm1)
            self.ffn_layers.append(ffn)
            self.norm2_layers.append(norm2)

        # prediction heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, training=False):
        # inputs shape: (batch, seq_len, n_assets)
        x = self.input_proj(inputs)
        x += self.pos_encoding[:, :self.seq_len, :]  # add positional encoding

        # encoder
        for attn, norm1, ffn, norm2 in zip(self.attn_layers, self.norm1_layers, self.ffn_layers, self.norm2_layers):
            attn_out = attn(x, x)
            x = norm1(x + attn_out)
            ffn_out = ffn(x)
            x = norm2(x + ffn_out)

        # use last timestep
        last = x[:, -1, :]
        mean_out = self.mean_head(last)
        var_out = self.var_head(last)
        return {"mean_head": mean_out, "var_head": var_out}

    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)

# -------------------------
# Builder for environment
# -------------------------
def build_transformer_for_env(env, n_assets=18, seq_len=60, d_model=64, num_heads=4, dff=128, num_layers=2, dropout=0.1, model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        num_layers=num_layers,
        dropout=dropout
    )

    # dummy input to build model
    dummy_x = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    # optionally load weights
    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded Informer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights. Starting fresh: {e}")"""




# -------------------------
# Positional Encoding
# -------------------------
"""def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]           # (seq_len, 1)
    i = np.arange(d_model)[np.newaxis, :]             # (1, d_model)
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    # apply sin to even indices, cos to odd indices
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]       # (1, seq_len, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)

# -------------------------
# Standard Multi-Head Attention Layer
# -------------------------
class FullAttention(layers.Layer):
    def __init__(self, d_model, num_heads, dropout=0.1, name=None):
        super().__init__(name=name)
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dropout = layers.Dropout(dropout)
        self.norm = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=False):
        attn_out = self.mha(x, x)
        attn_out = self.dropout(attn_out, training=training)
        return self.norm(x + attn_out)  # residual + norm

# -------------------------
# Informer Encoder Layer
# -------------------------
class InformerEncoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super().__init__()
        self.attn = FullAttention(d_model, num_heads, dropout)
        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation="relu"),
            layers.Dense(d_model)
        ])
        self.dropout = layers.Dropout(dropout)
        self.norm = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training=False):
        x = self.attn(x, training=training)
        ffn_out = self.ffn(x)
        ffn_out = self.dropout(ffn_out, training=training)
        return self.norm(x + ffn_out)

# -------------------------
# Short-Sequence Informer with Positional Encoding
# -------------------------
class TransformerModel(tf.keras.Model):
    def __init__(self, n_assets, seq_len,
                 d_model=64, num_heads=4, dff=128, num_layers=2,
                 dropout=0.1, name="informer_short_pe"):
        super().__init__(name=name)
        self.n_assets = n_assets
        self.seq_len = seq_len
        self.d_model = d_model
        self.pos_encoding = get_positional_encoding(seq_len, d_model)

        # input projection
        self.input_proj = layers.Dense(d_model, name="input_projection")

        # encoder layers
        self.enc_layers = [InformerEncoderLayer(d_model, num_heads, dff, dropout)
                           for _ in range(num_layers)]

        # output heads
        self.mean_head = layers.Dense(n_assets, name="mean_head")
        self.var_head = layers.Dense(n_assets, activation="softplus", name="var_head")

    def call(self, inputs, training=False):
        x = self.input_proj(inputs)
        x += self.pos_encoding[:, :self.seq_len, :]  # add positional encoding

        for layer in self.enc_layers:
            x = layer(x, training=training)

        last = x[:, -1, :]
        mean_out = self.mean_head(last)
        var_out = self.var_head(last)
        return {"mean_head": mean_out, "var_head": var_out}

    def compile(self, optimizer="adam", loss=None, metrics=None, **kwargs):
        if loss is None:
            loss = {"mean_head": "mse", "var_head": "mse"}
        super().compile(optimizer=optimizer, loss=loss, metrics=metrics, **kwargs)

# -------------------------
# Builder wrapper
# -------------------------
def build_transformer_for_env(env, n_assets=18, seq_len=60,
                                    d_model=64, num_heads=4, dff=128,
                                    num_layers=2, dropout=0.1, model_path=None):
    model = TransformerModel(n_assets=n_assets, seq_len=seq_len,
                            d_model=d_model, num_heads=num_heads,
                            dff=dff, num_layers=num_layers, dropout=dropout)
    # build with dummy input
    dummy = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy)

    # compile
    model.compile(optimizer="adam",
                  loss={"mean_head": "mse", "var_head": "mse"},
                  metrics={"mean_head": "mae", "var_head": "mae"})
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded short-sequence Informer+PE weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights from {model_path}: {e}\n→ Starting fresh.")"""


class CryptoEnvironment:
    def __init__(self, prices='/content/drive/MyDrive/crypto_portfolionew.csv', transformer=None, seq_len=60, n_assets=18, online_train=True, patience=5, batch_size=16, model_path='best_transformer_online.weights.h5', asset_cols=None):
        self.prices = prices
        self.capital = 1000.0
        self.data = self.load_data()  # implement your load_data method earlier
        self.seq_len = seq_len
        self.n_assets = n_assets
        self.online_train = online_train
        self.patience = patience
        self.batch_size = batch_size
        self.model_path = model_path
        self.asset_cols = asset_cols or [
            '<CLOSE>_BANK','<CLOSE>_BMLT','<CLOSE>_DTIP','<CLOSE>_FOLD','<CLOSE>_GDIR',
            '<CLOSE>_MADN','<CLOSE>_MAPN','<CLOSE>_MSMI','<CLOSE>_NORI','<CLOSE>_OIMC',
            '<CLOSE>_PASN','<CLOSE>_PKLJ','<CLOSE>_PNES','<CLOSE>_PTAP','<CLOSE>_SFKZ',
            '<CLOSE>_SSAP','<CLOSE>_TAMN','<CLOSE>_TSBE'
        ]

        # attach or build transformer (single source of truth)
        if transformer is None:
            build_transformer_for_env(self, n_assets=self.n_assets, seq_len=self.seq_len, model_path=self.model_path)
        else:
            self.transformer_model = transformer
            # ensure built & compiled
            _ = self.transformer_model(tf.zeros((1, self.seq_len, self.n_assets)))
            self.transformer_model.compile(optimizer='adam', loss={'mean_head':'mse','var_head':'mse'})



        """if transformer_model is not None and os.path.exists(self.model_path):
           try:
              self.transformer_model.load_weights(self.model_path)
           except Exception:
                pass"""

    def load_data(self):
        data =  pd.read_csv(self.prices)
        try:
            data.index = data['Date']
            data = data.drop(columns = ['Date'])
        except:
            multi_cols = pd.MultiIndex.from_arrays([
                     data.columns,
                     data.iloc[0],
                     data.iloc[1]])
            data.columns = multi_cols

            # Remove the first two rows since they became the new header levels
            data=data.iloc[2:].reset_index(drop=True)
            data.columns = ['_'.join(map(str, col)).strip() for col in data.columns]
            data=data.set_index('Unnamed: 0_<COL11>_Date')
            data.columns = (data.columns.str.replace(r'[\d.]+', '', regex=True).str.replace(r'nan', '', case=False, regex=True).str.replace(r'_+', '_', regex=True).str.strip('_')
)
        return data

    def preprocess_state(self, state):

        state = state.replace([np.inf, -np.inf], np.nan)
        state = state.fillna(0)
        scaler = MinMaxScaler(feature_range=(0,1))
         # scaling dataset
        state = pd.DataFrame(scaler.fit_transform(state),index=state.index, columns=state.columns)
        return state

    def get_state(self, t, lookback, is_cov_matrix = False, is_raw_time_series = True):

        assert lookback <= t

        decision_making_state = self.data.iloc[t-lookback:t]
        decision_making_state = decision_making_state
        #set_trace()
        if is_cov_matrix:
            x = decision_making_state.cov()
            scaler =MinMaxScaler(feature_range=(0,1))
            # scaling dataset
            x =  pd.DataFrame(scaler.fit_transform(x),index=x.index, columns=x.columns)
            '''retsmean=decision_making_state.mean()
            rets12=decision_making_state-retsmean
            lenx=rets12.shape[0]
            leny=rets12.shape[1]
            for i in range(lenx):
                for j in range(leny):
                    if (rets12.iloc[i,j]>0):
                        rets12.iloc[i,j]=0
            semicov=((rets12.T.dot(rets12))/(lenx-1))
            x=semicov'''
            return x
        elif not is_cov_matrix and not is_raw_time_series:
            x = decision_making_state.cov()
            scaler =MinMaxScaler(feature_range=(0,1))
            # scaling dataset
            #x =  scaler.fit_transform(x)
            decision_making_state = self.data.iloc[t-lookback:t]
            xx = decision_making_state.pct_change().dropna()
            #xx =  scaler.fit_transform(xx)
            xxx =  scaler.fit_transform(np.dot(xx,x))
            xxx= pd.DataFrame(xxx,index=xx.index, columns=xx.columns)

            return xxx
        else:
            if is_raw_time_series:

                decision_making_state = self.data.iloc[t-lookback:t]
                decision_making_state= decision_making_state.apply(pd.to_numeric, errors='coerce').drop(columns=['Unnamed: 0'], errors='ignore').dropna()
                returnsren =decision_making_state.iloc[:, :95].pct_change()
                cols_to_replace = decision_making_state.columns[:95]
                decision_making_state[cols_to_replace] = returnsren[cols_to_replace].values
                col_to_move = "<CLOSE>_IDXS"
                close_cols = [col for col in decision_making_state.columns if col.startswith("<CLOSE>") and col != col_to_move]
                if close_cols:
                         last_close_index = decision_making_state.columns.get_loc(close_cols[-1])
                else:
                         last_close_index = 0

                cols = list(decision_making_state.columns)
                cols.remove(col_to_move)
                cols.insert(last_close_index + 1, col_to_move)
                decision_making_state = decision_making_state[cols]

            return self.preprocess_state(decision_making_state.dropna())


    def compute_BL_anchor(self, t, lookback, tau, delta):
        """
        Compute a Black-Litterman anchor weights vector using a rolling window ending at index t.
        Returns a long-only weights vector that sums to 1.
        """
        # ensure enough history
        start_idx = max(0, t - lookback)
        window = self.data.iloc[start_idx:t][['<CLOSE>_BANK','<CLOSE>_BMLT','<CLOSE>_DTIP',
                                                    '<CLOSE>_FOLD','<CLOSE>_GDIR','<CLOSE>_MADN',
                                                    '<CLOSE>_MAPN','<CLOSE>_MSMI','<CLOSE>_NORI',
                                                    '<CLOSE>_OIMC','<CLOSE>_PASN','<CLOSE>_PKLJ',
                                                    '<CLOSE>_PNES','<CLOSE>_PTAP','<CLOSE>_SFKZ',
                                                    '<CLOSE>_SSAP','<CLOSE>_TAMN','<CLOSE>_TSBE']].astype(float).pct_change().dropna()


        """if window.shape[0] < 5:
            # fallback to equal weights if not enough data
            n = self.data.shape[1]
            return np.ones(n) / n"""

        # robust covariance (Ledoit-Wolf shrinkage)
        try:
            lw = LedoitWolf().fit(window.values)
            Sigma = lw.covariance_
        except Exception:
            Sigma = window.cov().values

        n = Sigma.shape[0]

        # Market implied returns (simple reverse optimization using equal market weights)

        index_returns = self.data['<CLOSE>_IDXS'].astype(float).pct_change().dropna().values
        # align sizes
        min_len = min(len(window.values), len(index_returns))
        X = window.values[-min_len:]
        y = index_returns[-min_len:]

        # --- Step B: constrained regression (cvxpy) ---
        w = cp.Variable(n)
        objective = cp.Minimize(0.5*cp.sum_squares(X @ w - y))
        constraints = [cp.sum(w) == 1, w >= 0]
        prob = cp.Problem(objective, constraints)
        prob.solve()

        """if w.value is None:
            # fallback: equal weights
            market_weights = np.zeros(n) / n
        else:"""
        market_weights = np.array(w.value).flatten()

        #market_weights = np.ones(n) / n
        pi = delta * Sigma.dot(market_weights)   # implied equilibrium returns

        n_assets = 18  # 18 assets
        seq_len=60
        # Transformer prediction
        if self.transformer_model is not None and window.shape[0] >= self.seq_len:
            x_input = tf.convert_to_tensor(window.values[-self.seq_len:][np.newaxis, ...], dtype=tf.float32)
            #model = self.transformer_model(n_assets=n_assets, seq_len=seq_len)
            #outputs = model(x_input, training=False)
            outputs = self.transformer_model(x_input, training=False)
            mu_pred = outputs["mean_head"]
            var_pred = outputs["var_head"]

            Q = mu_pred.numpy().flatten()
            view_var = var_pred.numpy().flatten()
        """else:
            Q = window.mean().values
            view_var = window.var().values"""

        # P = identity (one view per asset)
        P = np.eye(n)

        # Build Omega (diagonal) from view variances (scaled)
        # Avoid zeros: use variance of returns * small factor
        #view_var = window.var().values
        # scale factor to ensure Omega not too large/small
        omega_scale = 0.1
        Omega = np.diag(np.maximum(view_var * omega_scale, 1e-8))

        # Black-Litterman posterior expected returns
        tauSigma = tau * Sigma
        # compute inverse parts safely with pseudo-inverse fallback
        try:
            inv_tauSigma = np.linalg.inv(tauSigma)
        except np.linalg.LinAlgError:
            inv_tauSigma = np.linalg.pinv(tauSigma)
        try:
            Omega_inv = np.linalg.inv(Omega)
        except np.linalg.LinAlgError:
            Omega_inv = np.linalg.pinv(Omega)

        A = inv_tauSigma + P.T @ Omega_inv @ P
        try:
            A_inv = np.linalg.inv(A)
        except np.linalg.LinAlgError:
            A_inv = np.linalg.pinv(A)

        right = inv_tauSigma @ pi + P.T @ Omega_inv @ Q
        mu_bl = A_inv @ right

        # Convert posterior returns to mean-variance weights: w = inv(delta*Sigma) * mu
        try:
            inv_deltaSigma = np.linalg.inv(delta * Sigma)
        except np.linalg.LinAlgError:
            inv_deltaSigma = np.linalg.pinv(delta * Sigma)
        raw_w = inv_deltaSigma @ mu_bl

        # Project to long-only simplex (non-negative, sum to 1)
        raw_w = np.maximum(raw_w, 0.0)
        s = raw_w.sum()
        """if s <= 0:
            # fallback equal weights
            w_bl = np.ones(n) / n
        else:"""
        w_bl = raw_w / s

        return w_bl


        # Online training step for Transformer
        # -------------------
    def online_train_transformer(env, t, lookback=250, epochs=3):
        # this function expects env.data to be a pandas DataFrame with same asset columns as used for training
        if env.transformer_model is None or t < lookback + env.seq_len:
            return

        # build window and sequences (same logic as make_transformer_data)
        start_idx = max(0, t - lookback - env.seq_len)
        end_idx = t
        asset_cols = env.asset_cols if hasattr(env, "asset_cols") else None
        if asset_cols is None:
            raise ValueError("env.asset_cols must be set for online_train_transformer.")

        window = env.data.iloc[start_idx:end_idx][asset_cols].astype(float).pct_change().dropna()
        if window.shape[0] < env.seq_len + 1:
            return

        X_seq, Y_mean, Y_var = [], [], []
        for i in range(len(window) - env.seq_len):
            x_i = window.values[i:i+env.seq_len]
            y_i = window.values[i+env.seq_len]
            X_seq.append(x_i)
            Y_mean.append(y_i)
            Y_var.append(np.var(window.values[i:i+env.seq_len], axis=0))

        X_seq = np.array(X_seq, dtype=np.float32)
        Y_mean = np.array(Y_mean, dtype=np.float32)
        Y_var = np.array(Y_var, dtype=np.float32)
        if len(X_seq) < 2:
            return

        ds = tf.data.Dataset.from_tensor_slices((X_seq, {'mean_head': Y_mean, 'var_head': Y_var})).batch(env.batch_size)

        early_stop = EarlyStopping(monitor='loss', patience=env.patience, restore_best_weights=True)
        checkpoint = ModelCheckpoint(env.model_path, monitor='loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)

        # compile & train
        env.transformer_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss={'mean_head':'mse','var_head':'mse'})
        env.transformer_model.fit(ds, epochs=epochs, callbacks=[early_stop, checkpoint], verbose=0)
        print("✅ Online transformer checkpoint saved?:", os.path.exists(env.model_path))


    def get_reward(self, action, action_t, reward_t,W,capital,name, alpha = 0.50,
                   bl_lookback=1000, tau=0.05, delta=2.5):
        def local_portfolio(returns, weights):
              weights = np.array(weights)
              rets = returns.mean()
              retsmean = returns.mean()
              rets12 = returns - retsmean
              lenx = rets12.shape[0]
              rf = 0.21 / 252

              rets12[rets12 > 0] = 0
              semicov = ((rets12.T.dot(rets12)) / (lenx - 1))

              # Portfolio metrics
              P_ret = rets
              P_vol = returns.std()
              P_semivol=semicov
              P_sharpe = (P_ret - rf) / P_vol
              P_sortino = (P_ret - rf) / P_semivol
              # === New Metrics ===
              ## 1. Calmar Ratio
              cumulative_returns = (returns + 1).cumprod()
              running_max = cumulative_returns.cummax()
              drawdown = (cumulative_returns - running_max) / running_max
              max_drawdown = drawdown.min()

              # daily → annualized compound return
              annualized_return = (cumulative_returns.iloc[-1]) ** (1 / len(cumulative_returns)) - 1
              calmar_ratio = (annualized_return-rf) / abs(max_drawdown) if max_drawdown != 0 else np.nan

              ## 2. Excess Return to Value-at-Risk (VaR)
              excess_return = returns.mean() - rf
              var_95 = np.percentile(returns, 5)
              excess_return_to_var = excess_return / abs(var_95) if var_95 != 0 else np.nan

              # 3. Excess Return to Conditional Value-at-Risk (CVaR)
              excess_return = returns.mean() - rf
              var_95 = np.percentile(returns, 5)
              cvar_95 = returns[returns <= var_95].mean()
              excess_return_to_cvar = excess_return / abs(cvar_95) if cvar_95 != 0 else np.nan

              return np.array([P_ret, P_vol,P_semivol, P_sharpe,P_sortino, calmar_ratio, excess_return_to_var,excess_return_to_cvar])

        """# Online training
        if self.online_train:
            self.online_train_transformer(action_t, lookback=bl_lookback)
         # Normalize DQN action"""
        w_action = np.array(action, dtype=float)
        w_action = np.maximum(w_action, 0.0)
        w_action = w_action / (w_action.sum() if w_action.sum() > 0 else len(w_action))
        """if w_action.sum() <= 0:
            # fallback to equal
            w_action = np.ones_like(w_action) / len(w_action)
        else:
            w_action = np.maximum(w_action, 0.0)
            w_action = w_action / w_action.sum()"""

        # Compute BL anchor at time action_t (using data ending at action_t)
        #try:

        w_bl = self.compute_BL_anchor(action_t, lookback=bl_lookback, tau=tau, delta=delta)
        #except Exception:
            # fallback equal weights
            #w_bl = np.ones_like(w_action) / len(w_action)

        # Blend anchor and action


        if name=='eq':
              w_final = w_action
        else:
              w_final = (1.0 - alpha) * w_bl + alpha * w_action
        w_final = np.maximum(w_final, 0.0)
        """if w_final.sum() <= 0:
            w_final = np.ones_like(w_final) / len(w_final)
        else:"""
        w_final = w_final / w_final.sum()





        # Transaction costs (approx) using your existing logic
        cost_changes = list(np.array(w_final) - np.array(W))
        cocd = [element * 0.00371 if element >= 0 else abs(element) * 0.0088 for element in cost_changes]
        cocod = sum(cocd)

        # Select the same price columns you used originally (safe select)
        cols = ['<CLOSE>_BANK','<CLOSE>_BMLT','<CLOSE>_DTIP',
                '<CLOSE>_FOLD','<CLOSE>_GDIR','<CLOSE>_MADN',
                '<CLOSE>_MAPN','<CLOSE>_MSMI','<CLOSE>_NORI',
                '<CLOSE>_OIMC','<CLOSE>_PASN','<CLOSE>_PKLJ',
                '<CLOSE>_PNES','<CLOSE>_PTAP','<CLOSE>_SFKZ',
                '<CLOSE>_SSAP','<CLOSE>_TAMN','<CLOSE>_TSBE']
        # clamp columns to those present
        cols_present = [c for c in cols if c in self.data.columns]
        if len(cols_present) == 0:
            raise ValueError("None of the expected price columns are in self.data")

        data_period = self.data.iloc[action_t:reward_t][cols_present].apply(pd.to_numeric, errors='coerce').dropna(how='all')
        if data_period.shape[0] < 2:
            # Not enough price rows, return zero reward fallback
            returns_series = pd.Series(dtype=float)
            sharpe_arr = np.array([0.0] * len(w_final))
            returns=returns_series
            sharpe=sharpe_arr
            W=list(w_final)
            return returns, sharpe, W, capital

        # Build portfolio value series via shares method (avoids cumulative product oddities)
        first_prices = data_period.iloc[0].values.astype(float)
        # avoid zero division
        first_prices[first_prices == 0] = 1e-8
        shares = (capital * w_final) / first_prices  # number of shares or units per asset

        # compute portfolio value across the window
        prices_matrix = data_period.values.astype(float)
        pos_values = prices_matrix * shares.reshape(1, -1)  # each row is date
        portfolio_values = pos_values.sum(axis=1)

        # cash leftover invested at rf
        rf = 0.21 / 252.0
        invested = (shares * first_prices).sum()
        cash0 = capital - invested
        # cash growth series
        cash_series = cash0 * ((1.0 + rf) ** np.arange(len(data_period)))
        total_series = portfolio_values + cash_series

        # apply transaction cost once at the start
        total_series[0] = total_series[0] - cocod * total_series[0]

        # compute returns series (daily)
        total_series = pd.Series(total_series, index=data_period.index)
        returns_series = total_series.pct_change().dropna()
        if returns_series.empty:
            sharpe = 0.0
        else:
            # excess return per period: subtract rf per period
            excess = returns_series - rf
            # annualize-like sharpe using daily stats
            try:
                sharpe = (excess.mean() / (returns_series.std() + 1e-12)) * np.sqrt(252)
            except Exception:
                sharpe = 0.0

        # match original shape behavior: return sharpe as repeated array if needed
        weights = w_final
        sharpe=local_portfolio(returns_series, weights)[3]
        sharpe_arr = np.array([sharpe] * len(self.data.columns))

        final_capital = float(total_series.iloc[-1])
        returns=returns_series
        sharpe=sharpe_arr
        W=list(w_final)
        capital=final_capital
        # return consistent signature with your code


        #rew = (data_period.values[-1] - data_period.values[0]) / data_period.values[0]

        # === New Metrics ===
        ## 1. Calmar Ratio
        """cumulative_returns = kll['port'] / kll['port'].iloc[0]
        running_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - running_max) / running_max
        max_drawdown = drawdown.min()
        annualized_return = (cumulative_returns.iloc[-1]) ** (1 / len(cumulative_returns)) - 1
        sharpe = (annualized_return - rf)/ abs(max_drawdown) if max_drawdown != 0 else np.nan
        sharpe = np.array([sharpe] * (len(self.data.columns)-191))

        ## 2. Excess Return to Value-at-Risk (VaR)
        excess_return = returns.mean() - rf
        var_95 = np.percentile(returns, 5)
        sharpe = excess_return / abs(var_95) if var_95 != 0 else np.nan
        sharpe = np.array([sharpe] * (len(self.data.columns)-191))"""
        return returns, sharpe, W, capital,



In [ ]:
import numpy as np
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D,GlobalMaxPooling1D, Dense, Dropout, Flatten, LSTM, LayerNormalization, MultiHeadAttention, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Permute, Softmax, Multiply
from tensorflow.keras.layers import Dense, Multiply, Lambda,Embedding
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten,
    LayerNormalization, MultiHeadAttention, Add
)
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import Layer
import random
from collections import deque
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model



"""class Attention(tf.keras.layers.Layer):
          def __init__(self, units):
              super(Attention, self).__init__()
              self.W = tf.keras.layers.Dense(units)
              self.V = tf.keras.layers.Dense(1)
          def call(self, inputs):
              # Compute attention scores
              score = tf.nn.tanh(self.W(inputs))
              attention_weights = tf.nn.softmax(self.V(score), axis=1)

              # Apply attention weights to input
              context_vector = attention_weights * inputs
              context_vector = tf.reduce_sum(context_vector, axis=1)

              return context_vector"""

class Agent:

    def __init__(self, portfolio_size, is_eval=False, allow_short=False, buffer_size=50000):
        self.portfolio_size = portfolio_size
        self.allow_short = allow_short
        self.input_shape = (199, 209)
        self.action_size = 3  # sit, buy, sell

        # bounded replay buffer (prevents Colab hanging)
        self.memory4replay = deque(maxlen=buffer_size)

        self.is_eval = is_eval

        self.alpha = 0.85
        self.gamma = 0.50
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

        self.model = self._model()

    """def _model(self):
        inputs = Input(shape=self.input_shape)

        x = Conv1D(32, kernel_size=2, activation="relu", padding="same")(inputs)
        x = MaxPooling1D(pool_size=2)(x)

        x = Conv1D(32, kernel_size=2, activation="relu", padding="same")(x)
        x = MaxPooling1D(pool_size=2)(x)

        x = Flatten()(x)
        x = Dense(32, activation='elu')(x)
        x = Dropout(0.4)(x)

        predictions = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size - 191)]
        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer='adam', loss='mse')
        return model"""

    """def _model(self):
            inputs = Input(shape=self.input_shape)

            x = Conv1D(64, kernel_size=2, activation="relu", padding="same")(inputs)
            x = MaxPooling1D(pool_size=2)(x)

            x = LSTM(64, return_sequences=False)(x)
            x = Dense(64, activation='elu')(x)
            x = Dropout(0.4)(x)

            predictions = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size-191)]
            model = Model(inputs=inputs, outputs=predictions)
            model.compile(optimizer='adam', loss='mse')
            return model"""

    def _transformer_encoder(self, x, head_size=4, num_heads=2, ff_dim=64, dropout=0.4):
            attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
            attn_output = Dropout(dropout)(attn_output)
            x = Add()([x, attn_output])
            x = LayerNormalization(epsilon=1e-6)(x)

            ffn = Dense(ff_dim, activation="relu")(x)
            ffn = Dense(x.shape[-1])(ffn)
            x = Add()([x, ffn])
            x = LayerNormalization(epsilon=1e-6)(x)
            return x

    class SinusoidalPositionalEncoding(tf.keras.layers.Layer):
            def __init__(self, max_len, d_model):
                super().__init__()
                pos = np.arange(max_len)[:, np.newaxis]
                i = np.arange(d_model)[np.newaxis, :]
                angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
                angle_rads = pos * angle_rates
                angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
                angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
                self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)

            def call(self, x):
                seq_len = tf.shape(x)[1]
                return x + self.pos_encoding[:, :seq_len, :]

    def _model(self):
            inputs = Input(shape=self.input_shape)

            x = Dense(64)(inputs)  # Project features to d_model dimension
            x = self.SinusoidalPositionalEncoding(199, 64)(x)
            x = self._transformer_encoder(x)

            x = GlobalMaxPooling1D()(x)
            x = Dense(64, activation='elu')(x)
            x = Dropout(0.4)(x)

            predictions = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size-191)]
            model = Model(inputs=inputs, outputs=predictions)
            model.compile(optimizer='adam', loss='mse')
            return model




    """def attention_block(self,x):
       # x: (batch, timesteps, features)
       attention = Dense(1, activation='tanh')(x)  # (batch, timesteps, 1)
       attention = Lambda(lambda a: tf.nn.softmax(a, axis=1))(attention)  # softmax over time
       x = Multiply()([x, attention])  # (batch, timesteps, features)
       return x

    def _model(self):
        inputs = Input(shape=self.input_shape)

        x = Conv1D(filters=128, kernel_size=2, activation='relu', padding='valid')(inputs)
        x = MaxPooling1D(pool_size=2, strides=1)(x)
        x = self.attention_block(x)

        x = Flatten()(x)
        x = Dense(128, activation='elu')(x)
        x = Dropout(0.5)(x)
        x = Dense(64, activation='elu')(x)
        x = Dropout(0.5)(x)

        predictions = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size-191)]

        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer='adam', loss='mse')
        return model"""








    """def _model(self):

        inputs = Input(shape=self.input_shape)
        #Flatten()
        #x = Flatten()(inputs)
        x=keras.layers.Conv1D(filters=252, kernel_size=2,
                      strides=1, padding="valid",
                      activation="relu")(inputs)
        #x =Flatten()(x)

        x=keras.layers.MaxPooling1D(pool_size=2,strides=1, padding="valid")(x)
        x=keras.layers.Conv1D(filters=252, kernel_size=2,
                      strides=1, padding="valid",
                      activation="relu")( inputs)
        x=keras.layers.MaxPooling1D(pool_size=2,strides=1, padding="valid")(x)
        x=keras.layers.Conv1D(filters=252, kernel_size=2,
                      strides=1, padding="valid",
                      activation="relu")( inputs)
        x=keras.layers.MaxPooling1D(pool_size=2,strides=1, padding="valid")(x)


        #x=keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True),
                              # backward_layer=tf.keras.layers.LSTM(64, activation='relu', return_sequences=True, go_backwards=True),
                              #merge_mode="concat")(x)
        #x=keras.layers.Conv1D(filters=32, kernel_size=2,
                      #strides=1, padding="valid",
                      #activation="relu")(x)
       # x=keras.layers.MaxPooling1D(pool_size=2,strides=2, padding="valid")(x)
        #Attention(64),
        #tf.keras.layers.Reshape((128,1)),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(126, return_sequences=True),
                                 #backward_layer=tf.keras.layers.LSTM(126, activation='relu', return_sequences=True, go_backwards=True),
                                 #merge_mode="concat"),
        x = Dense(256, activation='elu')(x)

        #
        #x=keras.layers.LSTM(256,activation='relu',return_sequences=False)(x)
        #x=keras.layers.Reshape((256,1))(x)
        #x=keras.layers.LSTM(30,activation='relu',return_sequences=False)(x)
        x = Dropout(0.5)(x)
        x = Dense(256, activation='elu')(x)
        x = Dropout(0.5)(x)
        x = Flatten()(x)

        predictions = []
        for i in range(self.portfolio_size-191):
            asset_dense = Dense(self.action_size, activation='linear')(x)
            predictions.append(asset_dense)

        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer='adam', loss='mse')
        return model"""

    """def _model(self):
           inputs = Input(shape=self.input_shape)

           x = Conv1D(filters=128, kernel_size=2, activation='relu', padding='valid')(inputs)
           x = MaxPooling1D(pool_size=2, strides=1)(x)
           x = LSTM(128, return_sequences=False)(x)

           x = Dense(128, activation='elu')(x)
           x = Dropout(0.5)(x)
           x = Dense(64, activation='elu')(x)
           x = Dropout(0.5)(x)

           x = Flatten()(x)
           predictions = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size-191)]

           model = Model(inputs=inputs, outputs=predictions)
           model.compile(optimizer='adam', loss='mse')
           return model"""


    """def _transformer_encoder(self, x, head_size=4, num_heads=4, ff_dim=8, dropout=0.5):
        # Multi-head attention block
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
        attn_output = Dropout(dropout)(attn_output)
        x = Add()([x, attn_output])
        x = LayerNormalization(epsilon=1e-6)(x)

        # Feed-forward block
        ffn = Dense(ff_dim, activation="relu")(x)
        ffn = Dense(x.shape[-1])(ffn)
        x = Add()([x, ffn])
        x = LayerNormalization(epsilon=1e-6)(x)
        return x


    def _model(self):
        inputs = Input(shape=self.input_shape)
        x = Conv1D(32, 3, activation="relu", padding="same")(inputs)
        x = Conv1D(16, 3, activation="relu", padding="same")(x)

        x = self._transformer_encoder(x)  # encoder with default config
        x = GlobalMaxPooling1D()(x)
        x = Flatten()(x)
        x = Dense(128, activation='elu')(x)
        x = Dropout(0.4)(x)
        x = Dense(64, activation='elu')(x)
        x = Dropout(0.4)(x)

        predictions = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size - 191)]

        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer='adam', loss='mse')
        return model"""





    """def attention_block(self,x):
       # x: (batch, timesteps, features)
       attention = Dense(1, activation='tanh')(x)  # (batch, timesteps, 1)
       attention = Lambda(lambda a: tf.nn.softmax(a, axis=1))(attention)  # softmax over time
       x = Multiply()([x, attention])  # (batch, timesteps, features)
       return x

    def _model(self):
        inputs = Input(shape=self.input_shape)

        x = Conv1D(filters=128, kernel_size=2, activation='relu', padding='valid')(inputs)
        x = MaxPooling1D(pool_size=2, strides=1)(x)
        x = self.attention_block(x)

        x = Flatten()(x)
        x = Dense(128, activation='elu')(x)
        x = Dropout(0.5)(x)
        x = Dense(64, activation='elu')(x)
        x = Dropout(0.5)(x)

        predictions = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size-191)]

        model = Model(inputs=inputs, outputs=predictions)
        model.compile(optimizer='adam', loss='mse')
        return model"""


    """class SinusoidalPositionalEncoding(Layer):
        def __init__(self, max_len, d_model):
            super().__init__()
            self.max_len = max_len
            self.d_model = d_model

            # Create positional encodings once at initialization
            pos = np.arange(max_len)[:, np.newaxis]
            i = np.arange(d_model)[np.newaxis, :]
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            angle_rads = pos * angle_rates

            # Apply sin to even indices and cos to odd indices
            angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
            angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

            # Convert to constant tensor
            self.pos_encoding = tf.constant(angle_rads[np.newaxis, ...], dtype=tf.float32)  # shape (1, max_len, d_model)

        def call(self, x):
            seq_len = tf.shape(x)[1]
            return x + self.pos_encoding[:, :seq_len, :]

    class PositionalEmbedding(Layer):
        def __init__(self, max_len, d_model):
            super().__init__()
            self.pos_embedding = Embedding(input_dim=max_len, output_dim=d_model)

        def call(self, x):
            positions = tf.range(start=0, limit=tf.shape(x)[1])
            pos_encoding = self.pos_embedding(positions)
            return x + pos_encoding

    def _transformer_encoder(self, x, head_size=16, num_heads=4, ff_dim=64, dropout=0.2):
        # Multi-head attention block
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
        attn_output = Dropout(dropout)(attn_output)
        x = Add()([x, attn_output])
        x = LayerNormalization(epsilon=1e-6)(x)

        # Feed-forward block
        ffn = Dense(ff_dim, activation="relu")(x)
        ffn = Dense(x.shape[-1])(ffn)
        x = Add()([x, ffn])
        x = LayerNormalization(epsilon=1e-6)(x)
        return x

    def _model(self):
        inputs = Input(shape=self.input_shape)  # (timesteps, features)

        # 1. CNN
        x = Conv1D(filters=32, kernel_size=2, activation='relu', padding='same')(inputs)
        x = MaxPooling1D(pool_size=2, strides=1)(x)

        # 2. LSTM
        x = LSTM(16, return_sequences=True)(x)


        sequence_length = self.input_shape[0]
        d_model = x.shape[-1]
        positions = tf.expand_dims(tf.range(start=0, limit=sequence_length, delta=1), 0)  # shape (1, time)
        pos_encoding = Embedding(input_dim=1000, output_dim=d_model)(positions)           # shape (1, time, d_model)
        x = x + pos_encoding

        #x = self.PositionalEmbedding(max_len=1000, d_model=x.shape[-1])(x)
        x =self.SinusoidalPositionalEncoding(max_len=1000, d_model=x.shape[-1])(x)


        # 3. Transformer encoder
        x = self._transformer_encoder(x)

        # 4. Dense layers
        x = Flatten()(x)
        x = Dense(32, activation='elu')(x)
        x = Dropout(0.2)(x)
        x = Dense(16, activation='elu')(x)
        x = Dropout(0.2)(x)

        # 5. Multi-head output (one head per asset)
        outputs = [Dense(self.action_size, activation='linear')(x) for _ in range(self.portfolio_size-191)]

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model"""



    def nn_pred_to_weights(self, pred, allow_short=False):
        weights = np.zeros(len(pred))
        raw_weights = np.argmax(pred, axis=-1)

        saved_min = None
        for e, r in enumerate(raw_weights):
            if r == 0:  # sit
                weights[e] = 0
            elif r == 1:  # buy
                weights[e] = np.abs(pred[e][0][r])
            else:       # sell
                weights[e] = -np.abs(pred[e][0][r])

        if not allow_short:
            weights += np.abs(np.min(weights))
            saved_min = np.abs(np.min(weights))
            saved_sum = np.sum(weights)
        else:
            saved_sum = np.sum(np.abs(weights))

        weights /= saved_sum
        return weights, saved_min, saved_sum

    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            w = np.random.normal(0, 1, size=(self.portfolio_size - 191,))
            saved_min = None
            if not self.allow_short:
                w += np.abs(np.min(w))
                saved_min = np.abs(np.min(w))
            saved_sum = np.sum(w)
            w /= saved_sum
            return w, saved_min, saved_sum

        pred = self.model.predict(np.expand_dims(state, 0), verbose=0)
        return self.nn_pred_to_weights(pred, self.allow_short)

    def remember(self, s, s_, action, reward, done):
        self.memory4replay.append((s, s_, action, reward, done))

    def expReplay(self, batch_size):
        if len(self.memory4replay) < batch_size:
            return

        minibatch = random.sample(self.memory4replay, batch_size)

        def weights_to_nn_preds_with_reward(action_weights, reward,
                                            Q_star=np.zeros((self.portfolio_size - 191, self.action_size))):
            Q = np.zeros((self.portfolio_size - 191, self.action_size))
            for i in range(self.portfolio_size - 191):
                if action_weights[i] == 0:
                    Q[i][0] = reward[i] + self.gamma * np.max(Q_star[i][0])
                elif action_weights[i] > 0:
                    Q[i][1] = reward[i] + self.gamma * np.max(Q_star[i][1])
                else:
                    Q[i][2] = reward[i] + self.gamma * np.max(Q_star[i][2])
            return Q

        def restore_Q_from_weights_and_stats(action):
            action_weights, action_min, action_sum = action[0], action[1], action[2]
            action_weights = action_weights * action_sum
            if action_min is not None:
                action_weights = action_weights - action_min
            return action_weights

        for (s, s_, action, reward, done) in minibatch:
            action_weights = restore_Q_from_weights_and_stats(action)
            Q_learned_value = weights_to_nn_preds_with_reward(action_weights, reward)
            s, s_ = s.values, s_.values

            if not done:
                Q_star = self.model.predict(np.expand_dims(s_, 0), verbose=0)
                Q_learned_value = weights_to_nn_preds_with_reward(action_weights, reward, np.squeeze(Q_star))

            Q_learned_value = [xi.reshape(1, -1) for xi in Q_learned_value]
            Q_current_value = self.model.predict(np.expand_dims(s, 0), verbose=0)
            Q = [np.add(a * (1 - self.alpha), q * self.alpha)
                 for a, q in zip(Q_current_value, Q_learned_value)]

            self.model.fit(np.expand_dims(s, 0), Q, epochs=1, verbose=0)

        # decay epsilon once per replay call
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
N_ASSETS = 209 #53
agent = Agent(N_ASSETS)
env = CryptoEnvironment()
window_size = 200
episode_count =1
batch_size = 32
rebalance_period = 5 #every 90 days weight change

In [ ]:
# @title
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import deque
from tensorflow.keras.callbacks import ModelCheckpoint


# -------------------------------
# ✅ EarlyStopping for RL
# -------------------------------
class EarlyStoppingRL:
    def __init__(self, monitor='reward', mode='max', patience=10, min_delta=1e-3, verbose=True):
        self.monitor = monitor
        self.mode = mode
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose

        self.best_score = -np.inf if mode == 'max' else np.inf
        self.wait = 0
        self.stopped_epoch = 0
        self.stop_training = False
        self.best_episode = 0

    def check(self, current_value, episode):
        if self.mode == 'max':
            improvement = current_value - self.best_score
        else:
            improvement = self.best_score - current_value

        if improvement > self.min_delta:
            self.best_score = current_value
            self.best_episode = episode
            self.wait = 0
            if self.verbose:
                print(f"→ {self.monitor} improved to {current_value:.4f} at episode {episode}")
        else:
            self.wait += 1
            if self.verbose:
                print(f"No significant improvement in {self.monitor} at episode {episode} (wait={self.wait})")

            if self.wait >= self.patience:
                self.stop_training = True
                self.stopped_epoch = episode
                if self.verbose:
                    print(f"✋ Early stopping triggered at episode {episode}. "
                          f"Best {self.monitor} = {self.best_score:.4f} at episode {self.best_episode}")
        return self.stop_training
 # -------------------------
# Builder wrapper (keeps same naming across train/eval)
# -------------------------
"""def build_transformer_for_env(env, n_assets=18, seq_len=60, d_model=64, num_heads=4, dff=128, num_layers=2, dropout=0.1, model_path=None):
    model = TransformerModel(n_assets=n_assets, seq_len=seq_len, d_model=d_model, num_heads=num_heads, dff=dff, num_layers=num_layers, dropout=dropout)
    # build with dummy input
    dummy = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy)
    # compile
    model.compile(optimizer='adam', loss={'mean_head': 'mse', 'var_head': 'mse'}, metrics={'mean_head':'mae','var_head':'mae'})
    env.transformer_model = model

    # Optionally load weights safely
    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded transformer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load transformer weights from {model_path}: {e}\n→ Continuing with freshly initialized transformer.")"""

# -------------------------
# Builder wrapper
# -------------------------
"""def build_transformer_for_env(env, n_assets=18, seq_len=60, node_features=1,
                                  d_model=64, num_heads=4, dff=128, num_layers=2,
                                  gcn_units=32, dropout=0.1, model_path=None):
    model = TransformerModel(n_assets=n_assets, seq_len=seq_len, node_features=node_features,
                           d_model=d_model, num_heads=num_heads, dff=dff,
                           num_layers=num_layers, gcn_units=gcn_units, dropout=dropout)
    dummy_x = tf.zeros((1, seq_len, n_assets, node_features))
    _ = model(dummy_x)  # build
    model.compile(optimizer="adam", loss={"mean_head": "mse", "var_head": "mse"},
                  metrics={"mean_head": "mae", "var_head": "mae"})
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded GCN+Transformer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights from {model_path}: {e}")"""


# -------------------------
# Builder wrapper for env (keeps naming, weight loading)
# -------------------------
"""def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              node_features=1,
                              gcn_units=[32, 32],
                              lstm_units=64,
                              tcn_filters=64,
                              tcn_kernel=3,
                              tcn_blocks=2,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        node_features=node_features,
        gcn_units=gcn_units,
        lstm_units=lstm_units,
        tcn_filters=tcn_filters,
        tcn_kernel=tcn_kernel,
        tcn_blocks=tcn_blocks
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets, node_features))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded GCN+LSTM+TCN weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights from {model_path}: {e}")"""



# -------------------------
# Builder wrapper for env
# -------------------------
"""def build_transformer_for_env(env,
                      n_assets=18,
                      seq_len=60,
                      node_features=1,
                      gcn_units=[32, 32],
                      model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        node_features=node_features,
        gcn_units=gcn_units
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets, node_features))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded GNN weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights (architecture mismatch). Starting fresh.")"""


# -------------------------
# Builder wrapper for env
# -------------------------
"""def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              tcn_filters=64,
                              tcn_kernel=3,
                              tcn_blocks=3,
                              dropout=0.1,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        tcn_filters=tcn_filters,
        tcn_kernel=tcn_kernel,
        tcn_blocks=tcn_blocks,
        dropout=dropout
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded TCN weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights (architecture mismatch). Starting fresh.")"""


# -------------------------
# Builder wrapper for env
# -------------------------
def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              tcn_filters=64,
                              tcn_kernel=3,
                              tcn_blocks=3,
                              lstm_units=64,
                              dropout=0.1,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        tcn_filters=tcn_filters,
        tcn_kernel=tcn_kernel,
        tcn_blocks=tcn_blocks,
        lstm_units=lstm_units,
        dropout=dropout
    )

    # Build with dummy input
    dummy_x = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded TCN+LSTM weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights (architecture mismatch). Starting fresh.")


# -------------------------
# Builder wrapper for env
# -------------------------
"""def build_transformer_for_env(env,
                              n_assets=18,
                              seq_len=60,
                              d_model=64,
                              num_heads=4,
                              dff=128,
                              num_layers=2,
                              dropout=0.1,
                              model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        num_layers=num_layers,
        dropout=dropout
    )

    dummy = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded Informer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load Informer weights from {model_path}: {e}\n→ Starting fresh.")"""

# -------------------------
# Builder for environment
# -------------------------
"""def build_transformer_for_env(env, n_assets=18, seq_len=60, d_model=64, num_heads=4, dff=128, num_layers=2, dropout=0.1, model_path=None):
    model = TransformerModel(
        n_assets=n_assets,
        seq_len=seq_len,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        num_layers=num_layers,
        dropout=dropout
    )

    # dummy input to build model
    dummy_x = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy_x)

    model.compile(
        optimizer="adam",
        loss={"mean_head": "mse", "var_head": "mse"},
        metrics={"mean_head": "mae", "var_head": "mae"}
    )
    env.transformer_model = model

    # optionally load weights
    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded Informer weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights. Starting fresh: {e}")"""


# -------------------------
# Builder wrapper
# -------------------------
"""def build_transformer_for_env(env, n_assets=18, seq_len=60,
                                    d_model=64, num_heads=4, dff=128,
                                    num_layers=2, dropout=0.1, model_path=None):
    model = TransformerModel(n_assets=n_assets, seq_len=seq_len,
                            d_model=d_model, num_heads=num_heads,
                            dff=dff, num_layers=num_layers, dropout=dropout)
    # build with dummy input
    dummy = tf.zeros((1, seq_len, n_assets))
    _ = model(dummy)

    # compile
    model.compile(optimizer="adam",
                  loss={"mean_head": "mse", "var_head": "mse"},
                  metrics={"mean_head": "mae", "var_head": "mae"})
    env.transformer_model = model

    if model_path and os.path.exists(model_path):
        try:
            model.load_weights(model_path)
            print(f"✅ Loaded short-sequence Informer+PE weights from {model_path}")
        except Exception as e:
            print(f"⚠️ Could not load weights from {model_path}: {e}\n→ Starting fresh.")"""


# -------------------------
# train_transformer with checkpoint (weights-only)
# -------------------------
def train_transformer(env, epochs=50, batch_size=16, save_transformer_path="best_transformer_online.weights.h5", validation_split=0.1, verbose=1):
    # env.X_train: (N, seq_len, n_assets)
    # env.y_train: dict {"mean_head": arr, "var_head": arr}
    if not hasattr(env, "X_train") or not hasattr(env, "y_train"):
        raise ValueError("env.X_train and env.y_train must be set before calling train_transformer.")

    if not isinstance(env.y_train, dict) or "mean_head" not in env.y_train or "var_head" not in env.y_train:
        raise ValueError("env.y_train must be dict with keys 'mean_head' and 'var_head'.")

    # ensure transformer built
    if not hasattr(env, "transformer_model") or env.transformer_model is None:
        build_transformer_for_env(env, n_assets=env.n_assets, seq_len=env.seq_len, model_path=None)

    # ensure compiled
    if not hasattr(env.transformer_model, 'optimizer') or env.transformer_model.optimizer is None:
        env.transformer_model.compile(optimizer='adam', loss={'mean_head':'mse', 'var_head':'mse'})

    # checkpoint callback (weights-only)
    checkpoint = ModelCheckpoint(save_transformer_path, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1)

    history = env.transformer_model.fit(
        env.X_train,
        [env.y_train['mean_head'], env.y_train['var_head']],
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=[checkpoint],
        verbose=verbose
    )

    print(f"✅ Transformer training finished. Best weights saved to: {save_transformer_path}")
    return history





def train_agent(env,
                agent,
                max_episodes=10,
                batch_size=32,
                rebalance_period=5,
                n_assets=18,
                seq_len=60,
                window_size=200,
                buffer_size=50_000,
                train_every=10,
                save_agent_path="best_agent.weights.h5",
                save_transformer_path="best_transformer_online.weights.h5"):

    # make sure transformer exists before training
    if not hasattr(env, "transformer_model") or env.transformer_model is None:
        build_transformer_for_env(env, n_assets=n_assets, seq_len=seq_len)

    # pre-train transformer ONCE
    train_transformer(env, epochs=3, batch_size=batch_size,
                      save_transformer_path=save_transformer_path)

    env.online_train = True

    # persistent replay buffer
    if not hasattr(agent.memory4replay, "maxlen"):
        agent.memory4replay = deque(maxlen=buffer_size)

    reward_list, returns_per_episode = [], []
    early_stop = EarlyStoppingRL(monitor='reward', mode='max',
                                 patience=5, min_delta=1e-3, verbose=True)

    if not hasattr(agent, "model") or agent.model is None:
        agent.model = agent._model()

    # TRAINING LOOP
    for e in range(max_episodes):
        agent.is_eval = False
        print(f"\n📘 Episode {e+1}/{max_episodes} — epsilon={agent.epsilon:.4f}")

        data_length = len(env.data)
        returns_history, returns_history_equal, rewards_history = [], [], []

        # random start
        start_idx = np.random.randint(window_size + 1,
                                      max((data_length - 750 - window_size - 1), window_size + 2))
        s = env.get_state(start_idx, window_size)

        # portfolio init
        W0, W0eq = np.zeros(n_assets), np.zeros(n_assets)
        capital, capitaleq = 1000.0, 1000.0

        step_count = 0
        for t in range(window_size, data_length - 750, rebalance_period):
            step_count += 1
            date1 = t - rebalance_period
            s_ = env.get_state(t, window_size)

            # agent action
            action = agent.act(s_)
            w_action = action[0] if isinstance(action, (tuple, list)) else action

            # rewards (daily expanded returns)
            weighted_returns, reward, W0, capital = env.get_reward(w_action, date1, t, W0, capital, 'rl')
            weighted_returns_equal, reward_equal, W0eq, capitaleq = env.get_reward(
                np.ones(n_assets)/n_assets, date1, t, W0eq, capitaleq, 'eq'
            )

            rewards_history.append(float(np.mean(reward)))
            returns_history.extend(weighted_returns)           # ✅ daily expand
            returns_history_equal.extend(weighted_returns_equal)

            done = (t == data_length)
            agent.memory4replay.append((s, s_, action, reward, done))

            if len(agent.memory4replay) >= batch_size and step_count % train_every == 0:
                agent.expReplay(batch_size)

            s = s_

        # --- episode end
        avg_reward = np.mean(rewards_history) if rewards_history else 0.0
        reward_list.append(avg_reward)
        cum_returns = np.array(returns_history).cumsum()
        returns_per_episode.append(cum_returns)

        print(f"Episode {e+1} → avg_reward={avg_reward:.6f}")

        # save models
        if avg_reward > early_stop.best_score:
            agent.model.save_weights(save_agent_path)
            env.transformer_model.save_weights(save_transformer_path)
            print(f"📦 Saved best models (episode {e+1}, reward={avg_reward:.6f})")

        # 🔹 train transformer ONCE per episode
        train_transformer(env, epochs=3, save_transformer_path=save_transformer_path)

        daily_cum_returns = np.repeat(cum_returns, rebalance_period)
        daily_cum_returns = daily_cum_returns[:750]  # truncate to exactly 750 days

        daily_cum_returns_equal = np.repeat(np.array(returns_history_equal).cumsum(), rebalance_period)
        daily_cum_returns_equal = daily_cum_returns_equal[:750]
        # plot last 250 days for visualization
        if (e+1) % 5 == 0 or e == max_episodes-1:
            plt.figure(figsize=(12, 2))
            plt.plot(daily_cum_returns, color="black", label="RL agent")
            plt.plot(daily_cum_returns_equal, color="grey", ls="--", label="Equal-weighted")
            plt.title(f"Episode {e+1} Last 750 Days Cumulative Return")
            plt.legend()
            plt.show()


        if early_stop.check(avg_reward, e):
            print("✅ Early stopping triggered; ending training loop.")
            break

    return reward_list, returns_per_episode



def evaluate_agent(env, agent, n_assets=18, seq_len=60, rebalance_period=5, window_size=200,
                   load_agent_path="best_agent.weights.h5",
                   load_transformer_path="best_transformer_online.weights.h5"):

    env.online_train = False
    agent.is_eval = True
    agent.epsilon = 0.0

    # reload agent
    agent.model = agent._model()
    if os.path.exists(load_agent_path):
        agent.model.load_weights(load_agent_path)
        print(f"✅ Loaded agent weights: {load_agent_path}")

    # rebuild transformer (same architecture as training)
    build_transformer_for_env(env, n_assets=n_assets, seq_len=seq_len)

    # init weights before load
    dummy_input = tf.zeros((1, seq_len, n_assets))
    _ = env.transformer_model(dummy_input)

    # load saved weights
    if os.path.exists(load_transformer_path):
        env.transformer_model.load_weights(load_transformer_path)
        print(f"✅ Loaded transformer weights: {load_transformer_path}")

    # --- run evaluation loop (last 250 days)
    start_test = len(env.data) - 750
    returns_rl, returns_equal = [], []

    W0_rl, W0_eq = np.zeros(n_assets), np.zeros(n_assets)
    capital_rl, capital_eq = 1000.0, 1000.0
    actions_equal, actions_rl = [], []

    for t in range(start_test, len(env.data), rebalance_period):
        state = env.get_state(t, window_size)
        action = agent.act(state)
        w_action = action[0] if isinstance(action, (tuple, list)) else action

        weighted_returns_rl, reward_rl, W0_rl, capital_rl = env.get_reward(w_action, t-rebalance_period, t, W0_rl, capital_rl, 'rl')
        weighted_returns_eq, reward_eq, W0_eq, capital_eq = env.get_reward(np.ones(n_assets)/n_assets, t-rebalance_period, t, W0_eq, capital_eq, 'eq')

        # expand daily
        returns_rl.extend(weighted_returns_rl)
        returns_equal.extend(weighted_returns_eq)

        for _ in range(len(weighted_returns_rl)):
            actions_equal.append(np.ones(n_assets)/n_assets)
            actions_rl.append(w_action)

    # truncate to exactly 250 days
    returns_rl = returns_rl[:750]
    returns_equal = returns_equal[:750]
    actions_equal = actions_equal[:750]
    actions_rl = actions_rl[:750]

    mean_rl = np.mean(returns_rl)
    mean_eq = np.mean(returns_equal)

    return returns_rl, returns_equal, mean_rl, mean_eq, actions_equal, actions_rl





In [ ]:
# 1️⃣ Define a helper function for multi-output transformer (mean + variance)
def make_transformer_data(df, seq_len=60, n_assets=18):
    """
    Convert env.data (price dataframe) into (X, {y_mean, y_var}) sequences
    for multi-output transformer training.
    - y_mean = next step returns
    - y_var = rolling variance over past seq_len steps
    """
    df = df.astype(float).pct_change().dropna()

    X, y_mean, y_var = [], [], []
    for i in range(len(df) - seq_len):
        x_i = df.iloc[i:i+seq_len].values
        y_i = df.iloc[i+seq_len].values      # mean target = next step returns
        var_i = np.var(x_i, axis=0)          # variance target = past seq_len var

        X.append(x_i)
        y_mean.append(y_i)
        y_var.append(var_i)

    X = np.array(X, dtype=np.float32)
    y_mean = np.array(y_mean, dtype=np.float32)
    y_var = np.array(y_var, dtype=np.float32)

    # return multi-output labels as dict
    return X, {"mean_head": y_mean, "var_head": y_var}

# 2️⃣ Build dataset and attach to env
asset_cols = [
    '<CLOSE>_BANK','<CLOSE>_BMLT','<CLOSE>_DTIP','<CLOSE>_FOLD','<CLOSE>_GDIR',
    '<CLOSE>_MADN','<CLOSE>_MAPN','<CLOSE>_MSMI','<CLOSE>_NORI','<CLOSE>_OIMC',
    '<CLOSE>_PASN','<CLOSE>_PKLJ','<CLOSE>_PNES','<CLOSE>_PTAP','<CLOSE>_SFKZ',
    '<CLOSE>_SSAP','<CLOSE>_TAMN','<CLOSE>_TSBE'
]

X_train, y_train = make_transformer_data(env.data[asset_cols], seq_len=60, n_assets=18)

# attach to env
env.X_train, env.y_train = X_train, y_train

# Train
rewards = train_agent(env, agent, max_episodes=5, batch_size=32, rebalance_period=5)

# Evaluate
result_rl, result_equal, mean_rl, mean_eq,actions_equal, actions_rl = evaluate_agent(env, agent)



In [ ]:
import pandas as pd
import numpy as np

# Slice data exactly like the RL environment
dataind = read_csv('/content/drive/MyDrive/indexnew.csv', index_col=0, header=2)
dataind = dataind.rename(columns={'Unnamed: 1': 'IDXS'})
dataind = dataind[len(env.data)-750:len(env.data)].reset_index(drop=True)

# Prepare rebalance chunks
rebalance_period = 5
fgh = [dataind['IDXS'][i:i+rebalance_period] for i in range(0, len(dataind), rebalance_period)]

# Compute returns per chunk (no NaN drop, first return = 0)
x111 = [((chunk - chunk.shift(1)).fillna(0) / chunk.shift(1).replace(0, np.nan)).fillna(0).values
        for chunk in fgh]

# Concatenate all returns
xxx = np.concatenate(x111)

# Align length exactly with RL results
xxx = xxx[:len(result_rl)]
print(len(xxx), len(result_rl))  # Should now match


In [ ]:
#result_equal_vis = [item for sublist in result_equal for item in sublist]
#result_rl_vis = [item for sublist in result_rl for item in sublist]
result_equal_vis = result_equal
result_rl_vis = result_rl


In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(np.array(result_equal_vis).cumsum(), label = 'Benchmark', color = 'grey',ls = '--')
plt.plot(np.array(result_rl_vis).cumsum(), label = 'Deep RL portfolio', color = 'black',ls = '-')
plt.plot(np.array(xxx).cumsum(), label = 'Index', color = 'green',ls = '-.')
df11133445566=pd.DataFrame(np.array(xxx).cumsum(),columns=['Index']).join(pd.DataFrame(np.array(result_equal_vis).cumsum(),columns=['Benchmark']))
df1113344556677=pd.DataFrame(np.array(result_rl_vis).cumsum(),columns=['Deep RL portfolio']).join(df11133445566)
for var in list(['Benchmark','Deep RL portfolio','Index']):
    plt.annotate('%0.2f' % df1113344556677[var].iloc[-1], xy=(1, df1113344556677[var].iloc[-1]), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points',size=10, va="center")
plt.xlabel('Date')
# Set the y axis label of the current axis.
plt.ylabel('Cumluative_Return')
# Set a title of the current axes.
plt.title('Comparison of Performance')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
import statsmodels.api as sm
from statsmodels import regression
rf=0.21
def sharpe(R):
    #r = np.diff(R)
    sr = (R.mean()*252-rf)/(R.std() * np.sqrt(252))
    return sr

def sortino_ratio(series):
    #series = np.diff(series)
    mean = series.mean() * 252
    std_neg = series[series<0].std() * np.sqrt(252)
    sor=(mean-rf)/std_neg
    return sor

def max_drawdown(return_series):
    comp_ret = pd.Series((return_series+1).cumprod())
    peak = comp_ret.expanding(min_periods=1).max()
    dd = (comp_ret/peak)-1
    return dd.min()

def print_stats(result, benchmark):

    sharpe_ratio = sharpe(np.array(result))
    sor = sortino_ratio(np.array(result))
    max_draw=max_drawdown(np.array(result))
    returns = np.mean(np.array(result)*252)
    volatility = np.std(np.array(result)*np.sqrt(252))
    calmars = (returns-rf)/abs(max_draw)

    X = benchmark
    y = result
    x = sm.add_constant(X)
    model = regression.linear_model.OLS(y, x).fit()
    alpha = model.params[0]*252
    beta = model.params[1]



    return np.round(np.array([returns, volatility, sharpe_ratio,sor,max_draw,calmars, alpha, beta]), 4).tolist()

In [ ]:
pd.DataFrame(result_equal_vis).to_csv('/content/drive/MyDrive/equal_three.csv')
pd.DataFrame(xxx).to_csv('/content/drive/MyDrive/index_three.csv')
pd.DataFrame(result_rl_vis).to_csv('/content/drive/MyDrive/tcn+lstm_BL_TRA_three.csv')

In [ ]:
rl_infcnn1= read_csv('/content/drive/MyDrive/informer_BL_cnn_one.csv',index_col=0)
rl_infcnn1.columns = ['Informer_BL_Cnn_One']
rl_infcnn2= read_csv('/content/drive/MyDrive/informer_BL_cnn_two.csv',index_col=0)
rl_infcnn2.columns = ['Informer_BL_Cnn_Two']
rl_infcnn3= read_csv('/content/drive/MyDrive/informer_BL_cnn_three.csv',index_col=0)
rl_infcnn3.columns = ['Informer_BL_Cnn_Three']
rl_inftra1= read_csv('/content/drive/MyDrive/informer_BL_TRA_one.csv',index_col=0)
rl_inftra1.columns = ['Informer_BL_Tra_One']
rl_inftra2= read_csv('/content/drive/MyDrive/informer_BL_TRA_two.csv',index_col=0)
rl_inftra2.columns = ['Informer_BL_Tra_Two']
rl_inftra3= read_csv('/content/drive/MyDrive/informer_BL_TRA_three.csv',index_col=0)
rl_inftra3.columns = ['Informer_BL_Tra_Three']

rl_tracnn1= read_csv('/content/drive/MyDrive/transformer_BL_cnn_one.csv',index_col=0)
rl_tracnn1.columns = ['Transformer_BL_Cnn_One']
rl_tracnn2= read_csv('/content/drive/MyDrive/transformer_BL_cnn_two.csv',index_col=0)
rl_tracnn2.columns = ['Transformer_BL_Cnn_Two']
rl_tracnn3= read_csv('/content/drive/MyDrive/transformer_BL_cnn_three.csv',index_col=0)
rl_tracnn3.columns = ['Transformer_BL_Cnn_Three']
rl_tratra1= read_csv('/content/drive/MyDrive/transformer_BL_TRA_one.csv',index_col=0)
rl_tratra1.columns = ['Transformer_BL_Tra_One']
rl_tratra2= read_csv('/content/drive/MyDrive/transformer_BL_TRA_two.csv',index_col=0)
rl_tratra2.columns = ['Transformer_BL_Tra_Two']
rl_tratra3= read_csv('/content/drive/MyDrive/transformer_BL_TRA_three.csv',index_col=0)
rl_tratra3.columns = ['Transformer_BL_Tra_Three']

rl_tragcncnn1= read_csv('/content/drive/MyDrive/transformer+gcn_BL_cnn_one.csv',index_col=0)
rl_tragcncnn1.columns = ['Transformer+Gcn_BL_Cnn_One']
rl_tragcncnn2= read_csv('/content/drive/MyDrive/transformer+gcn_BL_cnn_two.csv',index_col=0)
rl_tragcncnn2.columns = ['Transformer+Gcn_BL_Cnn_Two']
rl_tragcncnn3= read_csv('/content/drive/MyDrive/transformer+gcn_BL_cnn_three.csv',index_col=0)
rl_tragcncnn3.columns = ['Transformer+Gcn_BL_Cnn_Three']
rl_tragcntra1= read_csv('/content/drive/MyDrive/transformer+gcn_BL_TRA_one.csv',index_col=0)
rl_tragcntra1.columns = ['Transformer+Gcn_BL_Tra_One']
rl_tragcntra2= read_csv('/content/drive/MyDrive/transformer+gcn_BL_TRA_two.csv',index_col=0)
rl_tragcntra2.columns = ['Transformer+Gcn_BL_Tra_Two']
rl_tragcntra3= read_csv('/content/drive/MyDrive/transformer+gcn_BL_TRA_three.csv',index_col=0)
rl_tragcntra3.columns = ['Transformer+Gcn_BL_Tra_Three']

rl_gcnlstmtcncnn1= read_csv('/content/drive/MyDrive/gcn+lstm+tcn_BL_cnn_one.csv',index_col=0)
rl_gcnlstmtcncnn1.columns = ['Gcn+Lstm+Tcn_BL_Cnn_One']
rl_gcnlstmtcncnn2= read_csv('/content/drive/MyDrive/gcn+lstm+tcn_BL_cnn_two.csv',index_col=0)
rl_gcnlstmtcncnn2.columns = ['Gcn+Lstm+Tcn_BL_Cnn_Two']
rl_gcnlstmtcncnn3= read_csv('/content/drive/MyDrive/gcn+lstm+tcn_BL_cnn_three.csv',index_col=0)
rl_gcnlstmtcncnn3.columns = ['Gcn+Lstm+Tcn_BL_Cnn_Three']
rl_gcnlstmtcntra1= read_csv('/content/drive/MyDrive/gcn+lstm+tcn_BL_TRA_one.csv',index_col=0)
rl_gcnlstmtcntra1.columns = ['Gcn+Lstm+Tcn_BL_Tra_One']
rl_gcnlstmtcntra2= read_csv('/content/drive/MyDrive/gcn+lstm+tcn_BL_TRA_two.csv',index_col=0)
rl_gcnlstmtcntra2.columns = ['Gcn+Lstm+Tcn_BL_Tra_Two']
rl_gcnlstmtcntra3= read_csv('/content/drive/MyDrive/gcn+lstm+tcn_BL_TRA_three.csv',index_col=0)
rl_gcnlstmtcntra3.columns = ['Gcn+Lstm+Tcn_BL_Tra_Three']

rl_gnncnn1= read_csv('/content/drive/MyDrive/gnn_BL_cnn_one.csv',index_col=0)
rl_gnncnn1.columns = ['Gnn_BL_Cnn_One']
rl_gnncnn2= read_csv('/content/drive/MyDrive/gnn_BL_cnn_two.csv',index_col=0)
rl_gnncnn2.columns = ['Gnn_BL_Cnn_Two']
rl_gnncnn3= read_csv('/content/drive/MyDrive/gnn_BL_cnn_three.csv',index_col=0)
rl_gnncnn3.columns = ['Gnn_BL_Cnn_Three']
rl_gnntra1= read_csv('/content/drive/MyDrive/gnn_BL_TRA_one.csv',index_col=0)
rl_gnntra1.columns = ['Gnn_BL_Tra_One']
rl_gnntra2= read_csv('/content/drive/MyDrive/gnn_BL_TRA_two.csv',index_col=0)
rl_gnntra2.columns = ['Gnn_BL_Tra_Two']
rl_gnntra3= read_csv('/content/drive/MyDrive/gnn_BL_TRA_three.csv',index_col=0)
rl_gnntra3.columns = ['Gnn_BL_Tra_Three']

rl_tcncnn1= read_csv('/content/drive/MyDrive/tcn_BL_cnn_one.csv',index_col=0)
rl_tcncnn1.columns = ['Tcn_BL_Cnn_One']
rl_tcncnn2= read_csv('/content/drive/MyDrive/tcn_BL_cnn_two.csv',index_col=0)
rl_tcncnn2.columns = ['Tcn_BL_Cnn_Two']
rl_tcncnn3= read_csv('/content/drive/MyDrive/tcn_BL_cnn_three.csv',index_col=0)
rl_tcncnn3.columns = ['Tcn_BL_Cnn_Three']
rl_tcntra1= read_csv('/content/drive/MyDrive/tcn_BL_TRA_one.csv',index_col=0)
rl_tcntra1.columns = ['Tcn_BL_Tra_One']
rl_tcntra2= read_csv('/content/drive/MyDrive/tcn_BL_TRA_two.csv',index_col=0)
rl_tcntra2.columns = ['Tcn_BL_Tra_Two']
rl_tcntra3= read_csv('/content/drive/MyDrive/tcn_BL_TRA_three.csv',index_col=0)
rl_tcntra3.columns = ['Tcn_BL_Tra_Three']


rl_tcnlstmcnn1= read_csv('/content/drive/MyDrive/tcn+lstm_BL_cnn_one.csv',index_col=0)
rl_tcnlstmcnn1.columns = ['Tcn+Lstm_BL_Cnn_One']
rl_tcnlstmcnn2= read_csv('/content/drive/MyDrive/tcn+lstm_BL_cnn_two.csv',index_col=0)
rl_tcnlstmcnn2.columns = ['Tcn+Lstm_BL_Cnn_Two']
rl_tcnlstmcnn3= read_csv('/content/drive/MyDrive/tcn+lstm_BL_cnn_three.csv',index_col=0)
rl_tcnlstmcnn3.columns = ['Tcn+Lstm_BL_Cnn_Three']
rl_tcnlstmtra1= read_csv('/content/drive/MyDrive/tcn+lstm_BL_TRA_one.csv',index_col=0)
rl_tcnlstmtra1.columns = ['Tcn+Lstm_BL_Tra_One']
rl_tcnlstmtra2= read_csv('/content/drive/MyDrive/tcn+lstm_BL_TRA_two.csv',index_col=0)
rl_tcnlstmtra2.columns = ['Tcn+Lstm_BL_Tra_Two']
rl_tcnlstmtra3= read_csv('/content/drive/MyDrive/tcn+lstm_BL_TRA_three.csv',index_col=0)
rl_tcnlstmtra3.columns = ['Tcn+Lstm_BL_Tra_Three']


rl_index1= read_csv('/content/drive/MyDrive/index_one.csv',index_col=0)
rl_index1.columns = ['Index_One']
rl_equal1= read_csv('/content/drive/MyDrive/equal_one.csv',index_col=0)
rl_equal1.columns = ['Equal_One']

rl_index2= read_csv('/content/drive/MyDrive/index_two.csv',index_col=0)
rl_index2.columns = ['Index_Two']
rl_equal2= read_csv('/content/drive/MyDrive/equal_two.csv',index_col=0)
rl_equal2.columns = ['Equal_Two']

rl_index3= read_csv('/content/drive/MyDrive/index_three.csv',index_col=0)
rl_index3.columns = ['Index_Three']
rl_equal3= read_csv('/content/drive/MyDrive/equal_three.csv',index_col=0)
rl_equal3.columns = ['Equal_Three']

In [ ]:
vnp=pd.concat([rl_infcnn1.cumsum(),rl_tracnn1.cumsum(),rl_tragcncnn1.cumsum(),rl_gcnlstmtcncnn1.cumsum(),
               rl_gnncnn1.cumsum(),rl_tcncnn1.cumsum(),rl_tcnlstmcnn1.cumsum(),rl_index1.cumsum(),rl_equal1.cumsum()],axis=1)
plt.figure(figsize = (10, 5))
plt.plot(rl_infcnn1.cumsum(), label = 'Informer_BL_Cnn_One',ls = '--')
plt.plot(rl_tracnn1.cumsum(), label = 'Transformer_BL_Cnn_One',ls = '-')
plt.plot(rl_tragcncnn1.cumsum(), label = 'Transformer+Gcn_BL_Cnn_One',ls = '-.')
plt.plot(rl_gcnlstmtcncnn1.cumsum(), label = 'Gcn+Lstm+Tcn_BL_Cnn_One',ls = 'dashdot')
plt.plot(rl_gnncnn1.cumsum(), label = 'Gnn_BL_Cnn_One',ls = ':')
plt.plot(rl_tcncnn1.cumsum(), label = 'Tcn_BL_Cnn_One',ls = 'solid')
plt.plot(rl_tcnlstmcnn1.cumsum(), label = 'Tcn+Lstm_BL_Cnn_One',ls = 'dashed')
plt.plot(rl_index1.cumsum(), label = 'Index_One',ls = 'dotted')
plt.plot(rl_equal1.cumsum(), label = 'Equal_One')
for var in list(vnp.columns):
    plt.annotate('%0.2f' % vnp[var].iloc[-1], xy=(1, vnp[var].iloc[-1]), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points',size=10, va="center")
plt.xlabel('Date')
# Set the y axis label of the current axis.
plt.ylabel('Cumluative_Return')
# Set a title of the current axes.
plt.title('Comparison of Performance')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
print('Informer_BL_Cnn_One', print_stats(rl_infcnn1, rl_index1))
print('Transformer_BL_Cnn_One', print_stats(rl_tracnn1, rl_index1))
print('Transformer+Gcn_BL_Cnn_One', print_stats(rl_tragcncnn1, rl_index1))
print('Gcn+Lstm+Tcn_BL_Cnn_One', print_stats(rl_gcnlstmtcncnn1, rl_index1))
print('Gnn_BL_Cnn_One', print_stats(rl_gnncnn1, rl_index1))
print('Tcn_BL_Cnn_One', print_stats(rl_tcncnn1, rl_index1))
print('Tcn+Lstm_BL_Cnn_One', print_stats(rl_tcnlstmcnn1, rl_index1))
print('Equal_One', print_stats(rl_equal1, rl_index1))
print('Index_One', print_stats(rl_index1, rl_index1))

In [ ]:
# Importing library
import scipy.stats as stats

# pre holds the mileage before
# applying the different engine oil
pre = rl_tracnn1
# post holds the mileage after
# applying the different engine oil
post = rl_tcncnn1
# Performing the paired sample t-test
stats.ttest_rel(pre, post,alternative='less')

In [ ]:
vnp=pd.concat([rl_infcnn2.cumsum(),rl_tracnn2.cumsum(),rl_tragcncnn2.cumsum(),rl_gcnlstmtcncnn2.cumsum(),
               rl_gnncnn2.cumsum(),rl_tcncnn2.cumsum(),rl_tcnlstmcnn2.cumsum(),rl_index2.cumsum(),rl_equal2.cumsum()],axis=1)
plt.figure(figsize = (10, 5))
plt.plot(rl_infcnn2.cumsum(), label = 'Informer_BL_Cnn_Two',ls = '--')
plt.plot(rl_tracnn2.cumsum(), label = 'Transformer_BL_Cnn_Two',ls = '-')
plt.plot(rl_tragcncnn2.cumsum(), label = 'Transformer+Gcn_BL_Cnn_Two',ls = '-.')
plt.plot(rl_gcnlstmtcncnn2.cumsum(), label = 'Gcn+Lstm+Tcn_BL_Cnn_Two',ls = 'dashdot')
plt.plot(rl_gnncnn2.cumsum(), label = 'Gnn_BL_Cnn_Two',ls = ':')
plt.plot(rl_tcncnn2.cumsum(), label = 'Tcn_BL_Cnn_Two',ls = 'solid')
plt.plot(rl_tcnlstmcnn2.cumsum(), label = 'Tcn+Lstm_BL_Cnn_Two',ls = 'dashed')
plt.plot(rl_index2.cumsum(), label = 'Index_Two',ls = 'dotted')
plt.plot(rl_equal2.cumsum(), label = 'Equal_Two')
for var in list(vnp.columns):
    plt.annotate('%0.2f' % vnp[var].iloc[-1], xy=(1, vnp[var].iloc[-1]), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points',size=10, va="center")
plt.xlabel('Date')
# Set the y axis label of the current axis.
plt.ylabel('Cumluative_Return')
# Set a title of the current axes.
plt.title('Comparison of Performance')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
print('Informer_BL_Cnn_Two', print_stats(rl_infcnn2, rl_index2))
print('Transformer_BL_Cnn_Two', print_stats(rl_tracnn2, rl_index2))
print('Transformer+Gcn_BL_Cnn_Two', print_stats(rl_tragcncnn2, rl_index2))
print('Gcn+Lstm+Tcn_BL_Cnn_Two', print_stats(rl_gcnlstmtcncnn2, rl_index2))
print('Gnn_BL_Cnn_Two', print_stats(rl_gnncnn2, rl_index2))
print('Tcn_BL_Cnn_Two', print_stats(rl_tcncnn2, rl_index2))
print('Tcn+Lstm_BL_Cnn_Two', print_stats(rl_tcnlstmcnn2, rl_index2))
print('Equal_Two', print_stats(rl_equal2, rl_index2))
print('Index_Two', print_stats(rl_index2, rl_index2))

In [ ]:
# Importing library
import scipy.stats as stats

# pre holds the mileage before
# applying the different engine oil
pre = rl_tracnn2
# post holds the mileage after
# applying the different engine oil
post = rl_tcncnn2
# Performing the paired sample t-test
stats.ttest_rel(pre, post,alternative='less')

In [ ]:
vnp=pd.concat([rl_infcnn3.cumsum(),rl_tracnn3.cumsum(),rl_tragcncnn3.cumsum(),rl_gcnlstmtcncnn3.cumsum(),
               rl_gnncnn3.cumsum(),rl_tcncnn3.cumsum(),rl_tcnlstmcnn3.cumsum(),rl_index3.cumsum(),rl_equal3.cumsum()],axis=1)
plt.figure(figsize = (10, 5))
plt.plot(rl_infcnn3.cumsum(), label = 'Informer_BL_Cnn_Three',ls = '--')
plt.plot(rl_tracnn3.cumsum(), label = 'Transformer_BL_Cnn_Three',ls = '-')
plt.plot(rl_tragcncnn3.cumsum(), label = 'Transformer+Gcn_BL_Cnn_Three',ls = '-.')
plt.plot(rl_gcnlstmtcncnn3.cumsum(), label = 'Gcn+Lstm+Tcn_BL_Cnn_Three',ls = 'dashdot')
plt.plot(rl_gnncnn3.cumsum(), label = 'Gnn_BL_Cnn_Three',ls = ':')
plt.plot(rl_tcncnn3.cumsum(), label = 'Tcn_BL_Cnn_Three',ls = 'solid')
plt.plot(rl_tcnlstmcnn3.cumsum(), label = 'Tcn+Lstm_BL_Cnn_Three',ls = 'dashed')
plt.plot(rl_index3.cumsum(), label = 'Index_Three',ls = 'dotted')
plt.plot(rl_equal3.cumsum(), label = 'Equal_Three')
for var in list(vnp.columns):
    plt.annotate('%0.2f' % vnp[var].iloc[-1], xy=(1, vnp[var].iloc[-1]), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points',size=10, va="center")
plt.xlabel('Date')
# Set the y axis label of the current axis.
plt.ylabel('Cumluative_Return')
# Set a title of the current axes.
plt.title('Comparison of Performance')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
print('Informer_BL_Cnn_Three', print_stats(rl_infcnn3, rl_index3))
print('Transformer_BL_Cnn_Three', print_stats(rl_tracnn3, rl_index3))
print('Transformer+Gcn_BL_Cnn_Three', print_stats(rl_tragcncnn3, rl_index3))
print('Gcn+Lstm+Tcn_BL_Cnn_Three', print_stats(rl_gcnlstmtcncnn3, rl_index3))
print('Gnn_BL_Cnn_Three', print_stats(rl_gnncnn3, rl_index3))
print('Tcn_BL_Cnn_Three', print_stats(rl_tcncnn3, rl_index3))
print('Tcn+Lstm_BL_Cnn_Three', print_stats(rl_tcnlstmcnn3, rl_index3))
print('Equal_Three', print_stats(rl_equal3, rl_index3))
print('Index_Three', print_stats(rl_index3, rl_index3))

In [ ]:
# Importing library
import scipy.stats as stats

# pre holds the mileage before
# applying the different engine oil
pre = rl_tcncnn3
# post holds the mileage after
# applying the different engine oil
post = rl_tracnn3
# Performing the paired sample t-test
stats.ttest_rel(pre, post,alternative='less')

In [ ]:
vnp=pd.concat([rl_inftra1.cumsum(),rl_tratra1.cumsum(),rl_tragcntra1.cumsum(),rl_gcnlstmtcntra1.cumsum(),
               rl_gnntra1.cumsum(),rl_tcntra1.cumsum(),rl_tcnlstmtra1.cumsum(),rl_index1.cumsum(),rl_equal1.cumsum()],axis=1)
plt.figure(figsize = (10, 5))
plt.plot(rl_inftra1.cumsum(), label = 'Informer_BL_Tra_One',ls = '--')
plt.plot(rl_tratra1.cumsum(), label = 'Transformer_BL_Tra_One',ls = '-')
plt.plot(rl_tragcntra1.cumsum(), label = 'Transformer+Gcn_BL_Tra_One',ls = '-.')
plt.plot(rl_gcnlstmtcntra1.cumsum(), label = 'Gcn+Lstm+Tcn_BL_Tra_One',ls = 'dashdot')
plt.plot(rl_gnntra1.cumsum(), label = 'Gnn_BL_Tra_One',ls = ':')
plt.plot(rl_tcntra1.cumsum(), label = 'Tcn_BL_Tra_One',ls = 'solid')
plt.plot(rl_tcnlstmtra1.cumsum(), label = 'Tcn+Lstm_BL_Tra_One',ls = 'dashed')
plt.plot(rl_index1.cumsum(), label = 'Index_One',ls = 'dotted')
plt.plot(rl_equal1.cumsum(), label = 'Equal_One_Three',)
for var in list(vnp.columns):
    plt.annotate('%0.2f' % vnp[var].iloc[-1], xy=(1, vnp[var].iloc[-1]), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points',size=10, va="center")
plt.xlabel('Date')
# Set the y axis label of the current axis.
plt.ylabel('Cumluative_Return')
# Set a title of the current axes.
plt.title('Comparison of Performance')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
print('Informer_BL_Tra_One', print_stats(rl_inftra1, rl_index1))
print('Transformer_BL_Tra_One', print_stats(rl_tratra1, rl_index1))
print('Transformer+Gcn_BL_Tra_One', print_stats(rl_tragcntra1, rl_index1))
print('Gcn+Lstm+Tcn_BL_Tra_One', print_stats(rl_gcnlstmtcntra1, rl_index1))
print('Gnn_BL_Tra_One', print_stats(rl_gnntra1, rl_index1))
print('Tcn_BL_Tra_One', print_stats(rl_tcntra1, rl_index1))
print('Tcn+Lstm_BL_Tra_One', print_stats(rl_tcnlstmtra1, rl_index1))
print('Equal_One', print_stats(rl_equal1, rl_index1))
print('Index_One', print_stats(rl_index1, rl_index1))

In [ ]:
# Importing library
import scipy.stats as stats

# pre holds the mileage before
# applying the different engine oil
pre = rl_tcnlstmtra1
# post holds the mileage after
# applying the different engine oil
post = rl_tcntra1
# Performing the paired sample t-test
stats.ttest_rel(pre, post,alternative='less')

In [ ]:
vnp=pd.concat([rl_inftra2.cumsum(),rl_tratra2.cumsum(),rl_tragcntra2.cumsum(),rl_gcnlstmtcntra2.cumsum(),
               rl_gnntra2.cumsum(),rl_tcntra2.cumsum(),rl_tcnlstmtra2.cumsum(),rl_index2.cumsum(),rl_equal2.cumsum()],axis=1)
plt.figure(figsize = (10, 5))
plt.plot(rl_inftra2.cumsum(), label = 'Informer_BL_Tra_Two',ls = '--')
plt.plot(rl_tratra2.cumsum(), label = 'Transformer_BL_Tra_Two',ls = '-')
plt.plot(rl_tragcntra2.cumsum(), label = 'Transformer+Gcn_BL_Tra_Two',ls = '-.')
plt.plot(rl_gcnlstmtcntra2.cumsum(), label = 'Gcn+Lstm+Tcn_BL_Tra_Two',ls = 'dashdot')
plt.plot(rl_gnntra2.cumsum(), label = 'Gnn_BL_Tra_Two',ls = ':')
plt.plot(rl_tcntra2.cumsum(), label = 'Tcn_BL_Tra_Two',ls = 'solid')
plt.plot(rl_tcnlstmtra2.cumsum(), label = 'Tcn+Lstm_BL_Tra_Two',ls = 'dashed')
plt.plot(rl_index2.cumsum(), label = 'Index_Two',ls = 'dotted')
plt.plot(rl_equal2.cumsum(), label = 'Equal_Two_Three',)
for var in list(vnp.columns):
    plt.annotate('%0.2f' % vnp[var].iloc[-1], xy=(1, vnp[var].iloc[-1]), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points',size=10, va="center")
plt.xlabel('Date')
# Set the y axis label of the current axis.
plt.ylabel('Cumluative_Return')
# Set a title of the current axes.
plt.title('Comparison of Performance')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
print('Informer_BL_Tra_Two', print_stats(rl_inftra2, rl_index2))
print('Transformer_BL_Tra_Two', print_stats(rl_tratra2, rl_index2))
print('Transformer+Gcn_BL_Tra_Two', print_stats(rl_tragcntra2, rl_index2))
print('Gcn+Lstm+Tcn_BL_Tra_Two', print_stats(rl_gcnlstmtcntra2, rl_index2))
print('Gnn_BL_Tra_Two', print_stats(rl_gnntra2, rl_index2))
print('Tcn_BL_Tra_Two', print_stats(rl_tcntra2, rl_index2))
print('Tcn+Lstm_BL_Tra_Two', print_stats(rl_tcnlstmtra2, rl_index2))
print('Equal_Two', print_stats(rl_equal2, rl_index2))
print('Index_Two', print_stats(rl_index2, rl_index2))

In [ ]:
# Importing library
import scipy.stats as stats

# pre holds the mileage before
# applying the different engine oil
pre = rl_gnntra2
# post holds the mileage after
# applying the different engine oil
post = rl_tcnlstmtra2
# Performing the paired sample t-test
stats.ttest_rel(pre, post,alternative='less')

In [ ]:
vnp=pd.concat([rl_inftra3.cumsum(),rl_tratra3.cumsum(),rl_tragcntra3.cumsum(),rl_gcnlstmtcntra3.cumsum(),
               rl_gnntra3.cumsum(),rl_tcntra3.cumsum(),rl_tcnlstmtra3.cumsum(),rl_index3.cumsum(),rl_equal3.cumsum()],axis=1)
plt.figure(figsize = (10, 5))
plt.plot(rl_inftra3.cumsum(), label = 'Informer_BL_Tra_Three',ls = '--')
plt.plot(rl_tratra3.cumsum(), label = 'Transformer_BL_Tra_Three',ls = '-')
plt.plot(rl_tragcntra3.cumsum(), label = 'Transformer+Gcn_BL_Tra_Three',ls = '-.')
plt.plot(rl_gcnlstmtcntra3.cumsum(), label = 'Gcn+Lstm+Tcn_BL_Tra_Three',ls = 'dashdot')
plt.plot(rl_gnntra3.cumsum(), label = 'Gnn_BL_Tra_Three',ls = ':')
plt.plot(rl_tcntra3.cumsum(), label = 'Tcn_BL_Tra_Three',ls = 'solid')
plt.plot(rl_tcnlstmtra3.cumsum(), label = 'Tcn+Lstm_BL_Tra_Three',ls = 'dashed')
plt.plot(rl_index3.cumsum(), label = 'Index_Three',ls = 'dotted')
plt.plot(rl_equal3.cumsum(), label = 'Equal_Tra_Three',)
for var in list(vnp.columns):
    plt.annotate('%0.2f' % vnp[var].iloc[-1], xy=(1, vnp[var].iloc[-1]), xytext=(8, 0),
                 xycoords=('axes fraction', 'data'), textcoords='offset points',size=10, va="center")
plt.xlabel('Date')
# Set the y axis label of the current axis.
plt.ylabel('Cumluative_Return')
# Set a title of the current axes.
plt.title('Comparison of Performance')
# show a legend on the plot
plt.legend()
plt.show()

In [ ]:
print('Informer_BL_Tra_Three', print_stats(rl_inftra3, rl_index3))
print('Transformer_BL_Tra_Three', print_stats(rl_tratra3, rl_index3))
print('Transformer+Gcn_BL_Tra_Three', print_stats(rl_tragcntra3, rl_index3))
print('Gcn+Lstm+Tcn_BL_Tra_Three', print_stats(rl_gcnlstmtcntra3, rl_index3))
print('Gnn_BL_Tra_Three', print_stats(rl_gnntra3, rl_index3))
print('Tcn_BL_Tra_Three', print_stats(rl_tcntra3, rl_index3))
print('Tcn+Lstm_BL_Tra_Three', print_stats(rl_tcnlstmtra3, rl_index3))
print('Equal_Three', print_stats(rl_equal3, rl_index3))
print('Index_Three', print_stats(rl_index3, rl_index3))

In [ ]:
# Importing library
import scipy.stats as stats

# pre holds the mileage before
# applying the different engine oil
pre = rl_tcntra3
# post holds the mileage after
# applying the different engine oil
post = rl_tcnlstmtra3
# Performing the paired sample t-test
stats.ttest_rel(pre, post,alternative='less')

In [ ]:
import matplotlib.pyplot as plt

# Data
companies = ['Fars','Femeli','Foolad','Vabemelat','Nouri','Shasta','Parsan','VaGhadir',
             'Tapico','Shepna','VaOmid','Vamaaden','Remapna','Vabank','Sefars','Tipico','Vasapa','Vabshahr']
returns = [0.410,0.447,0.427,0.465,0.838,0.556,0.523,0.446,0.399,0.490,0.446,0.448,0.419,0.473,0.371,0.419,0.367,0.332]
risks = [0.279,0.298,0.307,0.319,0.380,0.362,0.324,0.297,0.322,0.362,0.235,0.321,0.357,0.288,0.326,0.270,0.434,0.302]
sharpe = [0.715,0.797,0.708,0.800,1.652,0.958,0.966,0.792,0.587,0.774,1.003,0.742,0.585,0.913,0.495,0.759,0.363,0.406]

# Plot
plt.figure(figsize=(12,8))
scatter = plt.scatter(risks, returns, c=sharpe, cmap='viridis', s=[s*200 for s in sharpe], alpha=0.8)
for i, company in enumerate(companies):
    plt.text(risks[i]+0.005, returns[i]+0.005, company, fontsize=9)
plt.xlabel('Annual Risk (Volatility)')
plt.ylabel('Average Annual Return')
plt.title('Risk vs Return of 18 Selected Iranian Companies')
cbar = plt.colorbar(scatter)
cbar.set_label('Sharpe Ratio')
plt.grid(True)
plt.show()
